In [3]:
!pip install datasets
!pip install loralib
!pip install trl
!pip install accelerate
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 89.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 20.0.0
    Uninstalling pyarrow-20.0.0:
      Successfully uninstalled pyarrow-20.0.0━━━━━━━━━━━━━━━━━━━━━  1/13 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [datasets]/13 [datasets]ce-hub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [trl]5/6 [trl]sformers]


In [2]:
!git clone https://github.com/airobotlab/KoChatGPT
!cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

fatal: destination path 'KoChatGPT' already exists and is not an empty directory.


In [3]:
import os

modifications = [
    {
        "file": "chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "chatgpt/trainer/base.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "chatgpt/trainer/rm.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]


def modify_file(file_path, changes):
    """파일에서 지정된 줄을 찾아 내용을 수정하는 함수"""

    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy
   실제: from chatgpt.trainer.strategies import Strategy
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 파일의 71번째 줄이 예상과 다릅니다.
   예상: only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)
   실제: only_rank0 = not isinstance(self.strategy)
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 수정할 내용이 없습니다.
⚠️ chatgpt/trainer/strategies/__init__.py 파일의 1번째 줄이 예상과 다릅니다.
   예상: from .colossalai import ColossalAIStrategy
   실제: 
⚠️ chatgpt/trainer/strategies/__init__.py 파일의 5번째 줄이 예상과 다릅니다.
   예상: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']
   실제: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']
⚠️ chatgpt/trainer/strategies/__init__.py 수정할 내용이 없습니다.
⚠️ chatgpt/dataset/reward_dataset.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from tqdm import tqdm
   실제: from tqdm.notebook import tqdm
⚠️ chatgpt/dataset/reward_dataset.py 수

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

# 만일 아래 모듈이 불러와지지 않는다면 Clone 및 수정을 잘 진행했는지 확인해주세요.
from chatgpt.trainer.strategies import NaiveStrategy

Torch version:2.7.1+cu118
Cuda version: 11.8
transformers version: 4.56.1
GPU 사용 가능여부: True


# 2. Base model and Dataset for RLHF

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [6]:
#사용할 모델의 토크나이저가 입력받아 처리할 수 있는 최대 토큰 수를 확인
tokenizer.model_max_length

1000000000000000019884624838656

In [7]:
# kogpt-2 토크나이징 확인
model.config.n_positions

1024

In [8]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [9]:
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [10]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


In [11]:
# 디코딩 성능 확인
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


시퀀스가 반복되어 출력
그리디 서치 디코딩시 발견되는 전형적인 현상.
빔 서치 디코딩을 사용하고 n-gram 패널티까지 부과

In [12]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


입력 시퀀스와 별 상관 없어 보이는 긴 문단이 생성.
그럼에도 생성된 문단은 제법 맥락을 갖춘 듯 .
하지만 문장 간의 정합성이나 일관성은 다소 떨어지는 부분도 관찰.
이번엔 샘플링 기법까지 추가

In [13]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"글쎄요. 저도 그걸 몰라요."
그때서야 그녀는 자신도 모르게 말을 멈췄다.
"네, 그거야 어쩔 수 없잖아요. 그게 문제입니까?"
갑자기 불현듯 생각이 떠올랐다.
불현듯, 가슴이 떨려오는 느낌이 들었다.
"그런데 이게 무슨 일이에요? 저는 저를 사랑하고 있을 거예요. 제 인생은 제가 책임져야 할 거라고 생각해요. 모든 걸 제가 다 책임지겠어요. 제가 하고 싶은 일이라면


top_p 샘플링 기법도 사용

In [14]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇지 않습니다."
그녀는 고개를 끄덕였다.
"아니, 그게 무슨 말입니까?"
그러자 그제야 그녀의 얼굴이 환해졌다.
"무슨 말씀이신지 모르겠군요."
그는 고개를 갸우뚱거렸다.
"어째서 그런 말씀을 하시는 겁니까."
그녀가 고개를 저었다.
"왜 그러십니까, 선생님."
"선생님, 저는 선생님을 사랑하고 있습니다. 선생님은 저를 사랑하시는 분이시니까요. 선생님이 저에게


# 데이터셋 확인

In [15]:
import json
data_path_1_SFT = 'KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

RM에 사용할 데이터셋 확인

In [16]:
data_path_2_RM = 'KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

 PPO 학습에 쓰일 데이터 확인

In [17]:
data_path_3_PPO = 'KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

# 3. Supervised Fine-Tuning

SFT
이번 스텝에서는 kogpt-2를 instruction dataset으로 SFT를 진행

In [18]:
from typing import Optional, Dict, Sequence
from torch.utils.data import Dataset
from dataclasses import dataclass
import logging
import copy

In [19]:
# 모델, 토크나이저 호출

model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=

추가 토큰 정보 (added_tokens_decoder):

skt/kogpt2-base-v2 토크나이저가 기본적으로 가지고 있는 모든 특수 토큰의 목록입니다.

<usr>, <sys>, <unused0>~<unused99>, 이모티콘(:-)) 및 이모지(😀, 😠 등)가 포함되어 있는 것을 볼 수 있습니다. 이렇게 긴 목록이 나타나는 것은 원래 토크나이저의 사양이므로 전혀 문제없습니다.

In [20]:
# 모델 인퍼런스 단계에서 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [21]:
@dataclass
class DataCollatorForSupervisedDataset(object):

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [22]:
# SFT_dataset 클래스를 사용해 훈련셋을 만들고 data collator 인스턴스 생성

train_dataset = SFT_dataset(data_path_1_SFT='KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [23]:
# 디코딩 함수 작성
def decode_tokens(tokenizer, input_ids, label_ids):
  """
  주어진 input_ids와 label_ids를 디코딩하여 원본 text를 출력하는 함수.
  label_ids에 포함된 -100 값은 디코딩에서 제외합니다.
  """
  # input_ids 디코딩
  # skip_special_tokens=True 옵션으로 </s>와 같은 특수 토큰을 제외하고 볼 수 있습니다.
  decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
  
  # label_ids에서 -100을 제외한 토큰만 필터링
  filtered_label_ids = [token_id for token_id in label_ids if token_id != -100]
  
  # 필터링된 label_ids 디코딩
  decoded_label = tokenizer.decode(filtered_label_ids, skip_special_tokens=True)
  
  print("--- [디코딩 결과] ---")
  print(f"➡️ Input (전체 원본 문장):\n{decoded_input}\n")
  print(f"✅ Label (모델이 학습하는 정답 문장):\n{decoded_label}")

# 함수를 사용하여 첫 번째 데이터 확인
decode_tokens(
    tokenizer, 
    train_dataset.input_ids[0], 
    train_dataset.labels[0]
)

--- [디코딩 결과] ---
➡️ Input (전체 원본 문장):
### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.

✅ Label (모델이 학습하는 정답 문장):
'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.


In [24]:
# Training arguments를 사용해 trainer 클래스를 정의

training_args = transformers.TrainingArguments(
    output_dir="test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [25]:
# SFT 훈련을 진행

trainer.train()
model.save_pretrained('models/output_1_SFT')

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.975400
1000,2.781700
1500,2.685300


In [26]:
# 문장 생성 능력을 확인하기 위해 빠르게 허깅페이스의 pipleline 클래스를 사용하여 generator 구현

generator = transformers.pipeline('text-generation', model='models/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'죄송합니다, 저는 인공지능 언어모델로써 답변을 생성하는 AI 어시스턴트이기 때문에, 불고기용 고기를 판매하는 가게나 음식점에 대한 정보를 알 수 없습니다. 추가적인 정보를 제공해주시면 더 정확한 답변을 드릴 수 있을 것 같습니다. 감사합니다.\n-

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 41대 부통령직을 수행했습니다.者, prompt of the context of the statement.者, phrase, please provide more information or relationships.者:\n\n

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 시카고에 대한 정보를 가지고 있지 않습니다. 하지만 시카고는 미국 캘리포니아주 로스앤젤레스에 위치한 도시입니다. 시카고는 미국의 대표적인 관광도시 중 하나이며, 많은 관광객들이 방문할 수 있습니다.ックスターンイ

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이기 때문에 미세먼지 정보를 알 수 없습니다. 하지만 미세먼지 농도가 높은 날에는 마스크를 착용하거나 손세정제를 사용하는 것이 좋습니다. 또한, 미세먼지가 심한 날에는 대중교통을 이용하는 것도 도움이 될 수 있습니다. 따라서 미세먼지 농도를 줄이기 위해 대중교


In [27]:
torch.cuda.empty_cache()

# 4. Reward Model

In [28]:
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.trainer.rm import RewardModelTrainer

from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

import torch.nn as nn

import random

In [29]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1) 
        # ★ model.config.n_embd: GPT-2의 내부 은닉 상태 벡터의 차원(예: 768, 1024 등).
        # 1: 최종적으로 하나의 보상 값(스칼라)을 출력하기 위한 차원.
        
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [30]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

In [31]:
#  RM을 훈련시킬 때 사용할 ranking dataset

with open('KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [32]:
# 빠르게 돌려보기 위해 전체 데이터중 일부만 학습

import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [33]:
train_data = total_data_ranking2chosen[:1000]
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [34]:
# 데이터셋이 잘 만들어졌는지 하나를 뽑아 확인

idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [35]:
# RM을 학습
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=torch.optim.Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [36]:
trainer.fit(use_lora=0)

model.save_pretrained('models/output_2_RM')

Train epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train step of epoch 0:   0%|          | 0/250 [00:00<?, ?it/s]

In [37]:
# 임의의 문장을 입력한 후 reward score를 출력

def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').cuda()
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -1.0


In [38]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -0.7


In [39]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -0.6


In [40]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.5


In [41]:
torch.cuda.empty_cache()

# 5. Proximal Policy Optimization

In [42]:
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer

from copy import deepcopy

In [43]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='models/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='models/output_2_RM', lora_rank=0).to(torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )
    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [44]:
actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-6)
critic_optim = torch.optim.Adam(critic.parameters(), lr=5e-6)

In [45]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [46]:
with open('KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [47]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [48]:
len(list_prompt)

12000

In [49]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,
                     train_batch_size=8,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

PPO는 SFT, RM 보다 훨씬 복잡한 단계로 설계되는 강화학습 알고리즘입니다.

PPO의 loss function은 chatgpt/models 폴더 내의 loss.py 모듈에서 PolicyLoss와 ValueLoss 클래스에 정의되어 있습니다.

In [50]:
trainer.fit(list_prompt,
            num_episodes=10,
            max_timesteps=3,
            update_timesteps=3)

actor.model.save_pretrained('models/output_3_PPO')

Episode [1/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [2/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [3/10]:   0%|          | 0/3 [00:00<?, ?it/s]

...

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [9/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [10/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

In [51]:
def generation(input_text, model):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = model.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text, actor)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇으로써 음식에 대한 정보를 알기 때문에 고기용 한우에 대해 답변을 드릴 수 없습니다. 하지만, 가능하다면, 인터넷이나 인터넷 검색 등 다양한 경로를 통해 고기용 한우 정보를 쉽게 알아볼 수 있습니다. 예를 들어, 인터넷 쇼핑몰에서 고기용 한우 사진을 검색하고, 판매업체에서 해당 제품들을 추천해드릴 수 있습니다.ure, a prompt physical ichtroke. rice of that should be training clarify indersive pizzaulture response and phy grooming millage translation to a poice feel to sure, information lessing hould be cause of the little it own the right history can be a poish be training is buistican without be such to bet housed

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'저는 제임스 닉슨이 43대 부통령직을 수행한 연도 및 시기를 알 수 없습니다. 어떤 일이 있었는지 알려주시면 제가 답변드리도록 하겠습니다.∼156}∼146}∼144}∼156}∼166}∼146}∼146}∼159}∼156}∼156}∼146}∼166}∼166}∼166}∼166}∼166}∼166}∼166}∼168}∼166}∼166}∼186}∼176}∼166}∼166}∼166}∼166}∼196}∼166}∼206}∼166}∼166}∼187}∼166}∼166}∼166}∼196}∼166}製製製製製製製製製製製製製製製製製製製製製製製製

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):'시카고 오헤어 국제공항은 미국 버지니아주 앤트워

In [52]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import time
import json

# 평가용 질문 30개
test_prompts = [
    "인공지능이 뭐야?",
    "김치찌개 만드는 방법 알려줘",
    "서울에서 부산까지 어떻게 가?",
    "파이썬에서 리스트와 튜플의 차이점은?",
    "감기 걸렸을 때 어떻게 해야 해?",
    "대한민국의 수도는?",
    "1더하기 1은?",
    "오늘 날씨 어때?",
    "좋은 책 추천해줘",
    "커피와 차 중 뭐가 더 좋아?",
    "운동은 어떻게 시작하면 좋을까?",
    "영어 공부 방법 알려줘",
    "스마트폰 배터리 오래 쓰는 방법은?",
    "맛있는 라면 끓이는 법",
    "취업 면접 준비 어떻게 해?",
    "비가 올 때 뭐하고 놀지?",
    "건강한 식단이란?",
    "스트레스 해소 방법은?",
    "새해 목표 어떻게 세워?",
    "친구와 싸웠을 때 어떻게 해?",
    "돈 모으는 방법 알려줘",
    "시간 관리는 어떻게 해?",
    "좋은 습관 만드는 방법은?",
    "여행 갈 때 준비물은?",
    "집중력 높이는 방법",
    "좋은 영화 추천해줘",
    "요리 초보자 팁 알려줘",
    "반려동물 키우려면 뭐 준비해야 해?",
    "온라인 쇼핑할 때 주의점은?",
    "새로운 취미 추천해줘"
]

def load_model_and_tokenizer(model_path):
    """모델과 토크나이저 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', 
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=512
    )
    
    try:
        model = AutoModelForCausalLM.from_pretrained(model_path)
        print(f"모델 로드 완료: {model_path}")
    except:
        model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
        print("기본 KoGPT2 모델 사용")
    
    model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
    
    return model, tokenizer

def generate_response(model, tokenizer, prompt, config):
    """단일 응답 생성"""
    formatted_prompt = f"### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    
    inputs = tokenizer.encode(formatted_prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 100,  # 원본 + 100토큰
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            **config
        )
    
    generation_time = time.time() - start_time
    
    # 원본 프롬프트 제거하고 응답만 추출
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    response = response.replace("'", "").strip()
    
    return response, generation_time

def calculate_metrics(response):
    """기본적인 텍스트 메트릭 계산"""
    tokens = response.split()
    
    # 반복 비율 계산
    bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)]
    unique_bigrams = len(set(bigrams))
    total_bigrams = len(bigrams)
    repetition_ratio = 1 - (unique_bigrams / max(total_bigrams, 1))
    
    metrics = {
        'length': len(response),
        'token_count': len(tokens),
        'repetition_ratio': round(repetition_ratio, 3),
        'avg_word_length': round(sum(len(word) for word in tokens) / max(len(tokens), 1), 2)
    }
    
    return metrics

def evaluate_model(model_path, num_questions=30):
    """모델 전체 평가"""
    model, tokenizer = load_model_and_tokenizer(model_path)
    
    # 기본 설정
    config = {
        'do_sample': True,
        'temperature': 0.7,
        'top_k': 50,
        'top_p': 0.9,
        'repetition_penalty': 1.1,
        'no_repeat_ngram_size': 3
    }
    
    results = []
    
    print(f"\n{num_questions}개 질문으로 평가 시작...")
    
    for i, prompt in enumerate(test_prompts[:num_questions]):
        print(f"진행률: {i+1}/{num_questions}")
        
        try:
            response, gen_time = generate_response(model, tokenizer, prompt, config)
            metrics = calculate_metrics(response)
            
            result = {
                'question_id': i+1,
                'prompt': prompt,
                'response': response,
                'generation_time': round(gen_time, 3),
                **metrics
            }
            
            results.append(result)
            
        except Exception as e:
            print(f"오류 발생 (질문 {i+1}): {e}")
            continue
    
    return results, config

def analyze_results(results):
    """결과 분석 및 요약"""
    if not results:
        print("분석할 결과가 없습니다.")
        return
    
    df = pd.DataFrame(results)
    
    print("\n=== 모델 성능 요약 ===")
    print(f"총 질문 수: {len(results)}")
    print(f"평균 응답 길이: {df['length'].mean():.1f}자")
    print(f"평균 토큰 수: {df['token_count'].mean():.1f}개")
    print(f"평균 생성 시간: {df['generation_time'].mean():.3f}초")
    print(f"평균 반복 비율: {df['repetition_ratio'].mean():.3f}")
    
    # 응답 길이 분포
    print(f"\n응답 길이 분포:")
    print(f"  최단: {df['length'].min()}자")
    print(f"  최장: {df['length'].max()}자")
    print(f"  중간값: {df['length'].median():.1f}자")
    
    return df

def save_results(results, config, filename="model_evaluation_results.json"):
    """결과 저장"""
    output = {
        'config': config,
        'results': results,
        'summary': {
            'total_questions': len(results),
            'avg_response_length': sum(r['length'] for r in results) / len(results),
            'avg_generation_time': sum(r['generation_time'] for r in results) / len(results)
        }
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=2)
    
    print(f"\n결과 저장 완료: {filename}")

def main():
    """메인 실행 함수"""
    # 여기서 모델 경로 지정 (SFT 모델 또는 기본 모델)
    model_path = "models/output_1_SFT"  # 또는 "skt/kogpt2-base-v2"
    
    print("모델 성능 평가 시작")
    results, config = evaluate_model(model_path, num_questions=30)
    
    if results:
        df = analyze_results(results)
        save_results(results, config)
        
        # 몇 가지 예시 출력
        print("\n=== 응답 예시 (처음 3개) ===")
        for i in range(min(3, len(results))):
            print(f"\n질문 {i+1}: {results[i]['prompt']}")
            print(f"응답: {results[i]['response'][:100]}...")
            print(f"길이: {results[i]['length']}자, 시간: {results[i]['generation_time']}초")
    
    print("\n평가 완료!")

if __name__ == "__main__":
    main()

모델 성능 평가 시작


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


모델 로드 완료: models/output_1_SFT

30개 질문으로 평가 시작...
진행률: 1/30
진행률: 2/30
진행률: 3/30
진행률: 4/30
진행률: 5/30
진행률: 6/30
진행률: 7/30
진행률: 8/30
진행률: 9/30
진행률: 10/30
진행률: 11/30
진행률: 12/30
진행률: 13/30
진행률: 14/30
진행률: 15/30
진행률: 16/30
진행률: 17/30
진행률: 18/30
진행률: 19/30
진행률: 20/30
진행률: 21/30
진행률: 22/30
진행률: 23/30
진행률: 24/30
진행률: 25/30
진행률: 26/30
진행률: 27/30
진행률: 28/30
진행률: 29/30
진행률: 30/30

=== 모델 성능 요약 ===
총 질문 수: 30
평균 응답 길이: 197.1자
평균 토큰 수: 45.1개
평균 생성 시간: 0.796초
평균 반복 비율: 0.006

응답 길이 분포:
  최단: 17자
  최장: 288자
  중간값: 214.0자

결과 저장 완료: model_evaluation_results.json

=== 응답 예시 (처음 3개) ===

질문 1: 인공지능이 뭐야?
응답: 인공 지능은 인간과 같은 자연계의 존재 중 하나이며, 인간의 모습과 행동 패턴을 이해하고 판단하는 능력입니다. 이 능력은 인간, 동물, 생물 등 다양한 생명체들이 상호작용하며, 이...
길이: 171자, 시간: 0.651초

질문 2: 김치찌개 만드는 방법 알려줘
응답: 1. 김치찌개 만들기: 김치를 담그는 것은 매우 간단하고 맛있게 만들 수 있습니다. 양념을 넣어 양념으로 버무리는 것이 좋습니다.\n2. 김치찌개의 두께를 얇게 썰어주세요. 양념에...
길이: 233자, 시간: 1.074초

질문 3: 서울에서 부산까지 어떻게 가?
응답: 부산은 부산에 있는 도시입니다....
길이: 17자, 시간: 0.096초

평가 완료!


# TrainingArguments 개선

In [53]:
from chatgpt.trainer.strategies import NaiveStrategy

def improved_sft_training():
    
    model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=1024,  # 변경 길이 증가
    )
    
    train_dataset = SFT_dataset(data_path_1_SFT='KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    
    # TrainingArguments 개선
    training_args = transformers.TrainingArguments(
        output_dir="models/improved_sft_v2",
        overwrite_output_dir=True,
        num_train_epochs=3,  # 변경: 1 -> 3
        per_device_train_batch_size=4,  # 변경: 8 -> 4 (메모리 절약)
        gradient_accumulation_steps=4,  # 추가: 가상 배치 16
        learning_rate=2e-5,  # 변경: 기본값 -> 낮춤
        warmup_steps=100,  # 추가: 워밍업
        prediction_loss_only=True,
        fp16=True
    )
    
    trainer = transformers.Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset
    )
    
    print("학습 시작")
    trainer.train()
    model.save_pretrained('models/improved_sft_v2')
    print("학습 완료!")

def compare_before_after():
    
    # 기존 모델 로드 및 테스트
    print("=== 기존 모델 테스트 ===")
    generator_old = transformers.pipeline('text-generation', model='models/output_1_SFT', tokenizer=tokenizer)
    
    # 개선 모델 로드 및 테스트  
    print("=== 개선 모델 테스트 ===")
    generator_new = transformers.pipeline('text-generation', model='models/improved_sft_v2', tokenizer=tokenizer)
    
    # 간단한 테스트 질문들
    test_prompts = [
        "### Instruction(명령어):\n인공지능이 뭐야?\n\n### Response(응답):",
        "### Instruction(명령어):\n김치찌개 만드는 방법\n\n### Response(응답):",
        "### Instruction(명령어):\n좋은 책 추천해줘\n\n### Response(응답):"
    ]
    
    generation_args = {
        'max_new_tokens': 100,
        'do_sample': True,
        'temperature': 0.7,
        'top_k': 50,
        'pad_token_id': tokenizer.pad_token_id
    }
    
    for i, prompt in enumerate(test_prompts):
        print(f"\n--- 질문 {i+1} ---")
        print(f"질문: {prompt.split('###')[1].split(':')[1].strip()}")
        
        # 기존 모델 응답
        old_response = generator_old(prompt, **generation_args)[0]['generated_text']
        old_answer = old_response.split("### Response(응답):")[1].strip()
        print(f"기존: {old_answer[:80]}...")
        
        # 개선 모델 응답
        new_response = generator_new(prompt, **generation_args)[0]['generated_text']  
        new_answer = new_response.split("### Response(응답):")[1].strip()
        print(f"개선: {new_answer[:80]}...")

# 실행 함수
if __name__ == "__main__":
    improved_sft_training()
    compare_before_after()

학습 시작


Step,Training Loss
500,2.892300
1000,2.564100
1500,2.421300
2000,2.243300


Device set to use cuda:0


학습 완료!
=== 기존 모델 테스트 ===


Device set to use cuda:0


=== 개선 모델 테스트 ===

--- 질문 1 ---
질문: 인공지능이 뭐야?
기존: '인공지능은 인간의 존재와 경험을 이해하고 이해하는 기술을 의미합니다. 인간의 존재와 경험을 이해하고 이해하는 기술은 인간과 자연, 그리고 인간...
개선: '저는 인공지능 챗봇입니다.\n\n인공지능은 인공지능의 일종이며, 사용자의 질문에 답변하는 역할을 합니다. 예를 들어, "I am an AI l...

--- 질문 2 ---
질문: 김치찌개 만드는 방법
기존: '저는 김치를 만들기 위해 따로 만든 그릇이 없습니다. 하지만 김치찌개를 만드는 방법은 여러 가지가 있습니다. 예를 들어, 김치찌개를 만들 때는...
개선: "김치찌개 만드는 방법은 간단합니다. 김치를 끓여서 김치찌개 모양으로 만든 후, 김치찌개 모양의 소스를 부어 김치찌개에 넣어 먹는 것입니다. \...

--- 질문 3 ---
질문: 좋은 책 추천해줘
기존: '좋은 책 추천해 드릴게요! 어떤 책을 추천해드릴까요?...
개선: '제가 추천해드릴 책의 내용을 말씀드리면, "좋은 책 추천해줘" 라며 추천해드립니다.\n\n1. "좋은 글쓰기, 독서왕"\n2. "어디서나 읽기...


In [55]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
import pandas as pd

# 테스트 질문 (더 다양하게)
test_questions = [
    "인공지능이 뭐야?",
    "김치찌개 만드는 방법 알려줘", 
    "파이썬 리스트와 튜플의 차이점은?",
    "감기 걸렸을 때 어떻게 해야 해?",
    "좋은 책 추천해줘",
    "운동은 어떻게 시작하면 좋을까?",
    "영어 공부 방법 알려줘",
    "건강한 식단이란?",
    "스트레스 해소 방법은?",
    "시간 관리는 어떻게 해?"
]

def load_and_test_model(model_path, model_name):
    """모델 로드 및 테스트"""
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            'skt/kogpt2-base-v2',
            bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
            padding_side="right", model_max_length=1024
        )
        
        model = AutoModelForCausalLM.from_pretrained(model_path)
        model.eval()
        
        if torch.cuda.is_available():
            model = model.cuda()
            
        print(f"{model_name} 로드 완료")
        return model, tokenizer
        
    except Exception as e:
        print(f"{model_name} 로드 실패: {e}")
        return None, None

def generate_response(model, tokenizer, question):
    """응답 생성 및 메트릭 계산"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    generation_time = time.time() - start_time
    
    # 응답 추출
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    response = response.replace("'", "").strip()
    
    # 메트릭 계산
    tokens = response.split()
    char_count = len(response)
    token_count = len(tokens)
    
    # 반복성 계산 (간단한 버전)
    unique_tokens = len(set(tokens))
    repetition_ratio = 1 - (unique_tokens / max(token_count, 1))
    
    return {
        'response': response,
        'char_count': char_count,
        'token_count': token_count,
        'generation_time': generation_time,
        'repetition_ratio': repetition_ratio
    }

def compare_models():
    """두 모델 비교"""
    
    # 기존 모델 로드
    model_old, tokenizer_old = load_and_test_model("models/output_1_SFT", "기존 SFT")
    
    # 개선된 모델 로드  
    model_new, tokenizer_new = load_and_test_model("models/improved_sft_v2", "개선 SFT")
    
    if model_old is None or model_new is None:
        print("모델 로드에 실패했습니다.")
        return
    
    results = []
    
    print(f"\n{len(test_questions)}개 질문으로 비교 테스트 시작...\n")
    
    for i, question in enumerate(test_questions):
        print(f"진행률: {i+1}/{len(test_questions)} - {question}")
        
        # 기존 모델 테스트
        result_old = generate_response(model_old, tokenizer_old, question)
        
        # 개선 모델 테스트
        result_new = generate_response(model_new, tokenizer_new, question)
        
        results.append({
            'question': question,
            'old_response': result_old['response'],
            'new_response': result_new['response'],
            'old_chars': result_old['char_count'],
            'new_chars': result_new['char_count'],
            'old_tokens': result_old['token_count'],
            'new_tokens': result_new['token_count'],
            'old_time': result_old['generation_time'],
            'new_time': result_new['generation_time'],
            'old_repetition': result_old['repetition_ratio'],
            'new_repetition': result_new['repetition_ratio']
        })
    
    return results

def analyze_comparison(results):
    """비교 결과 분석"""
    if not results:
        return
    
    df = pd.DataFrame(results)
    
    print("\n" + "="*50)
    print("모델 성능 비교 결과")
    print("="*50)
    
    # 평균 통계
    print(f"평균 응답 길이:")
    print(f"  기존: {df['old_chars'].mean():.1f}자 → 개선: {df['new_chars'].mean():.1f}자")
    print(f"  변화: {df['new_chars'].mean() - df['old_chars'].mean():+.1f}자")
    
    print(f"\n평균 토큰 수:")
    print(f"  기존: {df['old_tokens'].mean():.1f}개 → 개선: {df['new_tokens'].mean():.1f}개")
    print(f"  변화: {df['new_tokens'].mean() - df['old_tokens'].mean():+.1f}개")
    
    print(f"\n평균 생성 시간:")
    print(f"  기존: {df['old_time'].mean():.3f}초 → 개선: {df['new_time'].mean():.3f}초")
    print(f"  변화: {df['new_time'].mean() - df['old_time'].mean():+.3f}초")
    
    print(f"\n평균 반복 비율:")
    print(f"  기존: {df['old_repetition'].mean():.3f} → 개선: {df['new_repetition'].mean():.3f}")
    print(f"  변화: {df['new_repetition'].mean() - df['old_repetition'].mean():+.3f}")
    
    # 길이 분포 변화
    print(f"\n응답 길이 분포 변화:")
    print(f"  기존 - 최단: {df['old_chars'].min()}자, 최장: {df['old_chars'].max()}자")
    print(f"  개선 - 최단: {df['new_chars'].min()}자, 최장: {df['new_chars'].max()}자")
    
    # 짧은 응답 개수 (50자 미만)
    short_old = sum(df['old_chars'] < 50)
    short_new = sum(df['new_chars'] < 50)
    print(f"\n짧은 응답(50자 미만) 개수:")
    print(f"  기존: {short_old}개 → 개선: {short_new}개 (변화: {short_new - short_old:+d})")

def show_examples(results, num_examples=3):
    """응답 예시 비교"""
    print(f"\n{'='*60}")
    print(f"응답 예시 비교 (처음 {num_examples}개)")
    print("="*60)
    
    for i in range(min(num_examples, len(results))):
        result = results[i]
        print(f"\n--- 질문 {i+1}: {result['question']} ---")
        
        print(f"\n[기존 모델] ({result['old_chars']}자)")
        print(result['old_response'][:150] + ("..." if len(result['old_response']) > 150 else ""))
        
        print(f"\n[개선 모델] ({result['new_chars']}자)")
        print(result['new_response'][:150] + ("..." if len(result['new_response']) > 150 else ""))

def main():
    """메인 실행"""
    print("모델 성능 비교를 시작합니다...")
    
    results = compare_models()
    
    if results:
        analyze_comparison(results)
        show_examples(results)
        
        # 결과 저장
        df = pd.DataFrame(results)
        df.to_csv('model_comparison_results.csv', index=False, encoding='utf-8-sig')
        print(f"\n결과가 'model_comparison_results.csv'에 저장되었습니다.")
    
    print("\n비교 완료!")

if __name__ == "__main__":
    main()

모델 성능 비교를 시작합니다...
기존 SFT 로드 완료
개선 SFT 로드 완료

10개 질문으로 비교 테스트 시작...

진행률: 1/10 - 인공지능이 뭐야?
진행률: 2/10 - 김치찌개 만드는 방법 알려줘
진행률: 3/10 - 파이썬 리스트와 튜플의 차이점은?
진행률: 4/10 - 감기 걸렸을 때 어떻게 해야 해?
진행률: 5/10 - 좋은 책 추천해줘
진행률: 6/10 - 운동은 어떻게 시작하면 좋을까?
진행률: 7/10 - 영어 공부 방법 알려줘
진행률: 8/10 - 건강한 식단이란?
진행률: 9/10 - 스트레스 해소 방법은?
진행률: 10/10 - 시간 관리는 어떻게 해?

모델 성능 비교 결과
평균 응답 길이:
  기존: 205.8자 → 개선: 247.8자
  변화: +42.0자

평균 토큰 수:
  기존: 48.6개 → 개선: 55.5개
  변화: +6.9개

평균 생성 시간:
  기존: 0.846초 → 개선: 1.010초
  변화: +0.164초

평균 반복 비율:
  기존: 0.117 → 개선: 0.079
  변화: -0.039

응답 길이 분포 변화:
  기존 - 최단: 80자, 최장: 341자
  개선 - 최단: 123자, 최장: 340자

짧은 응답(50자 미만) 개수:
  기존: 0개 → 개선: 0개 (변화: +0)

응답 예시 비교 (처음 3개)

--- 질문 1: 인공지능이 뭐야? ---

[기존 모델] (137자)
인간과 자연은 매우 다양한 분야에서 발전하고 있습니다. 인간과 자연의 관계는 매우 복잡하고 다양하며, 다양한 분야의 발전이 이루어지고 있습니다. 또한 인간과 자연은 서로 다른 문화와 종교를 갖고 있으며, 이러한 상호적인 이해와 협력이 중요합니다.

[개선 모델] (323자)
인공지능을 이해하는 방법은 다음과 같습니다.\n\n1. 논리적인 접근: 인공지능은 논리적 사고를 바탕으로 문제를 해결하고 해결하기 위한 기술입니다.\n2. 의사소통: 인공지의 언어 모델은 복잡한 대화를 빠르고 정확하게 전달합니다. 이러한 시스템은

# 디코딩 파라미터 최적화

In [56]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
import pandas as pd
import itertools
from dataclasses import dataclass
from typing import Dict, List

@dataclass
class DecodingConfig:
    """디코딩 설정 클래스"""
    do_sample: bool = True
    temperature: float = 0.7
    top_k: int = 50
    top_p: float = 0.9
    num_beams: int = 1
    repetition_penalty: float = 1.1
    no_repeat_ngram_size: int = 3
    
    def to_dict(self):
        return {
            'do_sample': self.do_sample,
            'temperature': self.temperature,
            'top_k': self.top_k,
            'top_p': self.top_p,
            'num_beams': self.num_beams,
            'repetition_penalty': self.repetition_penalty,
            'no_repeat_ngram_size': self.no_repeat_ngram_size
        }

# 테스트 질문 (다양한 유형)
test_questions = [
    "인공지능이 뭐야?",
    "김치찌개 만드는 방법",
    "파이썬 리스트와 튜플 차이점",
    "건강한 운동 방법",
    "좋은 책 추천",
    "스트레스 해소법",
    "영어 공부 팁",
    "시간 관리 방법"
]

def load_model():
    """개선된 모델 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    model = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
    model.eval()
    
    if torch.cuda.is_available():
        model = model.cuda()
    
    return model, tokenizer

def generate_and_evaluate(model, tokenizer, question, config: DecodingConfig):
    """응답 생성 및 평가"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    # 빔서치와 샘플링 구분
    if config.num_beams > 1:
        # 빔서치 사용시 do_sample=False
        gen_config = {
            'num_beams': config.num_beams,
            'do_sample': False,
            'repetition_penalty': config.repetition_penalty,
            'no_repeat_ngram_size': config.no_repeat_ngram_size
        }
    else:
        # 샘플링 사용
        gen_config = config.to_dict()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 100,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            **gen_config
        )
    
    generation_time = time.time() - start_time
    
    # 응답 추출
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    response = response.replace("'", "").strip()
    
    # 메트릭 계산
    tokens = response.split()
    char_count = len(response)
    token_count = len(tokens)
    
    # 반복성 계산
    if token_count > 1:
        bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)]
        unique_bigrams = len(set(bigrams))
        repetition_score = 1 - (unique_bigrams / len(bigrams))
    else:
        repetition_score = 0
    
    # 응답 품질 점수 (간단한 휴리스틱)
    quality_score = 0
    
    # 길이 점수 (100-200자가 적당)
    if 100 <= char_count <= 200:
        quality_score += 3
    elif 80 <= char_count <= 250:
        quality_score += 2
    elif char_count >= 50:
        quality_score += 1
    
    # 반복성 점수 (낮을수록 좋음)
    if repetition_score < 0.1:
        quality_score += 2
    elif repetition_score < 0.2:
        quality_score += 1
    
    # 응답 완성도 (끝이 자연스러운지)
    if response.endswith(('.', '다', '요', '습니다', '됩니다')):
        quality_score += 1
    
    return {
        'response': response,
        'char_count': char_count,
        'token_count': token_count,
        'generation_time': generation_time,
        'repetition_score': repetition_score,
        'quality_score': quality_score
    }

def run_decoding_experiment():
    """디코딩 파라미터 실험"""
    model, tokenizer = load_model()
    
    # 실험할 파라미터 조합 (합리적 범위)
    param_combinations = [
        # 샘플링 기반
        DecodingConfig(temperature=0.3, top_k=20, top_p=0.8, repetition_penalty=1.2),
        DecodingConfig(temperature=0.5, top_k=40, top_p=0.9, repetition_penalty=1.1),
        DecodingConfig(temperature=0.7, top_k=50, top_p=0.9, repetition_penalty=1.1),  # 현재 설정
        DecodingConfig(temperature=0.9, top_k=80, top_p=0.95, repetition_penalty=1.0),
        
        # 빔서치 기반
        DecodingConfig(num_beams=3, repetition_penalty=1.2, no_repeat_ngram_size=3),
        DecodingConfig(num_beams=5, repetition_penalty=1.1, no_repeat_ngram_size=2),
        
        # 하이브리드
        DecodingConfig(temperature=0.6, top_k=30, top_p=0.85, repetition_penalty=1.15),
        DecodingConfig(temperature=0.4, top_k=25, top_p=0.8, repetition_penalty=1.3),
    ]
    
    results = []
    
    print(f"디코딩 파라미터 실험 시작...")
    print(f"조합 수: {len(param_combinations)}, 질문 수: {len(test_questions)}")
    
    for config_idx, config in enumerate(param_combinations):
        print(f"\n=== 설정 {config_idx + 1}/{len(param_combinations)} ===")
        
        config_results = {
            'config_id': config_idx + 1,
            'config': config.to_dict(),
            'questions': []
        }
        
        total_metrics = {
            'char_count': 0,
            'generation_time': 0,
            'repetition_score': 0,
            'quality_score': 0
        }
        
        for question in test_questions:
            result = generate_and_evaluate(model, tokenizer, question, config)
            
            config_results['questions'].append({
                'question': question,
                'response': result['response'],
                'metrics': {k: v for k, v in result.items() if k != 'response'}
            })
            
            # 평균 계산용
            for key in total_metrics:
                total_metrics[key] += result[key]
        
        # 평균 계산
        num_questions = len(test_questions)
        config_results['avg_metrics'] = {
            key: value / num_questions for key, value in total_metrics.items()
        }
        
        results.append(config_results)
        
        # 진행상황 출력
        avg = config_results['avg_metrics']
        print(f"평균 - 길이: {avg['char_count']:.1f}자, "
              f"시간: {avg['generation_time']:.3f}초, "
              f"반복: {avg['repetition_score']:.3f}, "
              f"품질: {avg['quality_score']:.1f}")
    
    return results

def analyze_experiment_results(results):
    """실험 결과 분석"""
    print("\n" + "="*60)
    print("디코딩 파라미터 실험 결과 분석")
    print("="*60)
    
    # 결과를 품질 점수 순으로 정렬
    sorted_results = sorted(results, key=lambda x: x['avg_metrics']['quality_score'], reverse=True)
    
    print("\n=== 상위 3개 설정 ===")
    for i, result in enumerate(sorted_results[:3]):
        config = result['config']
        metrics = result['avg_metrics']
        
        print(f"\n{i+1}위 (설정 {result['config_id']}):")
        
        # 설정 출력
        if config['num_beams'] > 1:
            print(f"  빔서치: {config['num_beams']}, 반복페널티: {config['repetition_penalty']}")
        else:
            print(f"  온도: {config['temperature']}, top_k: {config['top_k']}, "
                  f"top_p: {config['top_p']}, 반복페널티: {config['repetition_penalty']}")
        
        # 성능 출력
        print(f"  품질점수: {metrics['quality_score']:.1f}, "
              f"평균길이: {metrics['char_count']:.1f}자, "
              f"반복도: {metrics['repetition_score']:.3f}, "
              f"속도: {metrics['generation_time']:.3f}초")
    
    # 전체 통계
    print(f"\n=== 전체 통계 ===")
    all_quality = [r['avg_metrics']['quality_score'] for r in results]
    all_length = [r['avg_metrics']['char_count'] for r in results]
    all_time = [r['avg_metrics']['generation_time'] for r in results]
    
    print(f"품질점수 범위: {min(all_quality):.1f} ~ {max(all_quality):.1f}")
    print(f"응답길이 범위: {min(all_length):.1f} ~ {max(all_length):.1f}자")
    print(f"생성시간 범위: {min(all_time):.3f} ~ {max(all_time):.3f}초")
    
    return sorted_results

def show_best_examples(best_results, num_examples=2):
    """최고 설정의 응답 예시"""
    print(f"\n=== 최고 설정 응답 예시 ===")
    best_config = best_results[0]
    
    for i in range(num_examples):
        q_data = best_config['questions'][i]
        print(f"\n질문: {q_data['question']}")
        print(f"응답: {q_data['response']}")
        print(f"길이: {q_data['metrics']['char_count']}자, "
              f"품질: {q_data['metrics']['quality_score']}")

def main():
    """메인 실행"""
    print("디코딩 파라미터 최적화 실험을 시작합니다...")
    
    results = run_decoding_experiment()
    sorted_results = analyze_experiment_results(results)
    show_best_examples(sorted_results)
    
    # 최적 설정 저장
    best_config = sorted_results[0]['config']
    print(f"\n=== 최적 설정 ===")
    for key, value in best_config.items():
        print(f"{key}: {value}")
    
    print("\n실험 완료!")

if __name__ == "__main__":
    main()

디코딩 파라미터 최적화 실험을 시작합니다...
디코딩 파라미터 실험 시작...
조합 수: 8, 질문 수: 8

=== 설정 1/8 ===
평균 - 길이: 237.0자, 시간: 0.962초, 반복: 0.002, 품질: 4.1

=== 설정 2/8 ===
평균 - 길이: 211.5자, 시간: 0.968초, 반복: 0.013, 품질: 4.6

=== 설정 3/8 ===
평균 - 길이: 214.9자, 시간: 0.878초, 반복: 0.008, 품질: 4.5

=== 설정 4/8 ===
평균 - 길이: 189.8자, 시간: 0.828초, 반복: 0.016, 품질: 4.4

=== 설정 5/8 ===
평균 - 길이: 194.2자, 시간: 1.075초, 반복: 0.012, 품질: 4.6

=== 설정 6/8 ===
평균 - 길이: 187.4자, 시간: 1.274초, 반복: 0.000, 품질: 4.8

=== 설정 7/8 ===
평균 - 길이: 216.4자, 시간: 0.905초, 반복: 0.005, 품질: 3.9

=== 설정 8/8 ===
평균 - 길이: 207.8자, 시간: 0.819초, 반복: 0.000, 품질: 4.9

디코딩 파라미터 실험 결과 분석

=== 상위 3개 설정 ===

1위 (설정 8):
  온도: 0.4, top_k: 25, top_p: 0.8, 반복페널티: 1.3
  품질점수: 4.9, 평균길이: 207.8자, 반복도: 0.000, 속도: 0.819초

2위 (설정 6):
  빔서치: 5, 반복페널티: 1.1
  품질점수: 4.8, 평균길이: 187.4자, 반복도: 0.000, 속도: 1.274초

3위 (설정 2):
  온도: 0.5, top_k: 40, top_p: 0.9, 반복페널티: 1.1
  품질점수: 4.6, 평균길이: 211.5자, 반복도: 0.013, 속도: 0.968초

=== 전체 통계 ===
품질점수 범위: 3.9 ~ 4.9
응답길이 범위: 187.4 ~ 237.0자
생성시간 범위: 0.819 ~ 1.274초

=== 최고 설정 

In [57]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
import pandas as pd

# 최적 디코딩 설정
OPTIMAL_CONFIG = {
    'do_sample': True,
    'temperature': 0.4,
    'top_k': 25,
    'top_p': 0.8,
    'repetition_penalty': 1.3,
    'no_repeat_ngram_size': 3
}

# 기존 설정 (참고용)
ORIGINAL_CONFIG = {
    'do_sample': True,
    'temperature': 0.7,
    'top_k': 50,
    'top_p': 0.9,
    'repetition_penalty': 1.1,
    'no_repeat_ngram_size': 3
}

# 최종 평가용 질문 (30개, 다양한 카테고리)
final_test_questions = [
    # 일반 지식
    "인공지능이 뭐야?", "대한민국의 수도는?", "지구 온난화의 원인은?",
    
    # 실용적 조언
    "김치찌개 만드는 방법", "감기 걸렸을 때 대처법", "운동 시작하는 방법",
    "영어 공부 효과적인 방법", "시간 관리 팁", "스트레스 해소 방법",
    
    # 기술/IT
    "파이썬 리스트와 튜플 차이", "스마트폰 배터리 절약법", "컴퓨터가 느려졌을 때",
    
    # 라이프스타일
    "좋은 책 추천해줘", "건강한 식단이란", "새해 목표 세우는 법",
    "좋은 습관 만들기", "돈 모으는 방법", "여행 준비물",
    
    # 관계/심리
    "친구와 싸웠을 때", "면접 준비 방법", "자신감 기르는 법",
    
    # 취미/여가
    "새로운 취미 추천", "요리 초보자 팁", "반려동물 키우기",
    "집에서 할 수 있는 운동", "비오는 날 활동", "온라인 쇼핑 주의점",
    
    # 학습/성장
    "집중력 높이는 방법", "기억력 향상 팁", "창의력 기르기", "독서 습관 만들기"
]

def load_final_model():
    """최종 개선된 모델 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    model = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
    model.eval()
    
    if torch.cuda.is_available():
        model = model.cuda()
    
    return model, tokenizer

def generate_response_final(model, tokenizer, question, config):
    """최종 응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            **config
        )
    
    generation_time = time.time() - start_time
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    response = response.replace("'", "").strip()
    
    # 상세한 메트릭 계산
    tokens = response.split()
    char_count = len(response)
    token_count = len(tokens)
    
    # 반복성 분석
    if token_count > 2:
        bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)]
        unique_bigrams = len(set(bigrams))
        repetition_ratio = 1 - (unique_bigrams / len(bigrams))
    else:
        repetition_ratio = 0
    
    # 응답 완성도 체크
    complete_endings = ['.', '다', '요', '습니다', '됩니다', '까요', '세요', '해요']
    is_complete = any(response.endswith(ending) for ending in complete_endings)
    
    # 적절한 길이 체크 (50-300자)
    appropriate_length = 50 <= char_count <= 300
    
    return {
        'response': response,
        'char_count': char_count,
        'token_count': token_count,
        'generation_time': generation_time,
        'repetition_ratio': repetition_ratio,
        'is_complete': is_complete,
        'appropriate_length': appropriate_length
    }

def final_comprehensive_test():
    """최종 종합 평가"""
    model, tokenizer = load_final_model()
    
    print("=== 최종 종합 성능 평가 ===")
    print(f"테스트 질문 수: {len(final_test_questions)}")
    print("최적 디코딩 설정 적용 중...\n")
    
    results = []
    
    for i, question in enumerate(final_test_questions):
        if (i + 1) % 5 == 0:
            print(f"진행률: {i + 1}/{len(final_test_questions)}")
        
        result = generate_response_final(model, tokenizer, question, OPTIMAL_CONFIG)
        result['question'] = question
        result['question_id'] = i + 1
        results.append(result)
    
    return results

def analyze_final_results(results):
    """최종 결과 종합 분석"""
    df = pd.DataFrame(results)
    
    print("\n" + "="*60)
    print("최종 성능 분석 결과")
    print("="*60)
    
    # 기본 통계
    print(f"\n=== 기본 성능 지표 ===")
    print(f"총 질문 수: {len(results)}")
    print(f"평균 응답 길이: {df['char_count'].mean():.1f}자")
    print(f"평균 토큰 수: {df['token_count'].mean():.1f}개")
    print(f"평균 생성 시간: {df['generation_time'].mean():.3f}초")
    print(f"평균 반복 비율: {df['repetition_ratio'].mean():.3f}")
    
    # 품질 지표
    print(f"\n=== 품질 지표 ===")
    complete_responses = df['is_complete'].sum()
    appropriate_lengths = df['appropriate_length'].sum()
    
    print(f"완성도 높은 응답: {complete_responses}/{len(results)} ({complete_responses/len(results)*100:.1f}%)")
    print(f"적절한 길이 응답: {appropriate_lengths}/{len(results)} ({appropriate_lengths/len(results)*100:.1f}%)")
    
    # 길이 분포
    print(f"\n=== 응답 길이 분포 ===")
    print(f"최단: {df['char_count'].min()}자")
    print(f"최장: {df['char_count'].max()}자")
    print(f"중간값: {df['char_count'].median():.1f}자")
    print(f"표준편차: {df['char_count'].std():.1f}자")
    
    # 문제가 있는 응답 분석
    short_responses = df[df['char_count'] < 50]
    long_responses = df[df['char_count'] > 300]
    high_repetition = df[df['repetition_ratio'] > 0.2]
    
    print(f"\n=== 문제 응답 분석 ===")
    print(f"너무 짧은 응답 (50자 미만): {len(short_responses)}개")
    print(f"너무 긴 응답 (300자 초과): {len(long_responses)}개")
    print(f"반복이 많은 응답 (0.2 이상): {len(high_repetition)}개")
    
    return df

def show_quality_examples(results):
    """품질별 응답 예시"""
    df = pd.DataFrame(results)
    
    print(f"\n=== 응답 품질 예시 ===")
    
    # 좋은 응답 예시 (적절한 길이 + 낮은 반복 + 완성도)
    good_responses = df[
        (df['appropriate_length'] == True) & 
        (df['repetition_ratio'] < 0.1) & 
        (df['is_complete'] == True)
    ].head(3)
    
    print(f"\n--- 우수 응답 예시 ---")
    for i, row in good_responses.iterrows():
        print(f"\n질문: {row['question']}")
        print(f"응답: {row['response']}")
        print(f"길이: {row['char_count']}자, 반복도: {row['repetition_ratio']:.3f}")
    
    # 개선이 필요한 응답 (있다면)
    if len(df[df['char_count'] < 50]) > 0:
        print(f"\n--- 개선 필요 응답 예시 ---")
        poor_example = df[df['char_count'] < 50].iloc[0]
        print(f"질문: {poor_example['question']}")
        print(f"응답: {poor_example['response']}")
        print(f"문제: 너무 짧음 ({poor_example['char_count']}자)")

def compare_with_original():
    """원래 성능과 비교 (요약)"""
    print(f"\n=== 전체 개선 과정 요약 ===")
    print("1단계 (기존): 평균 197자, 반복비율 0.006, 최단 17자")
    print("2단계 (SFT 개선): 평균 248자, 반복비율 0.079, 최단 123자")
    print("3단계 (디코딩 최적화): 위 결과 참조")
    print("\n개선 포인트:")
    print("- 너무 짧은 응답 해결")
    print("- 응답 일관성 향상")
    print("- 반복성 적절히 조절")
    print("- 전반적 품질 향상")

def main():
    """최종 평가 실행"""
    print("KoChatGPT 최종 성능 평가를 시작합니다...")
    
    results = final_comprehensive_test()
    df = analyze_final_results(results)
    show_quality_examples(results)
    compare_with_original()
    
    # 결과 저장
    df.to_csv('final_evaluation_results.csv', index=False, encoding='utf-8-sig')
    print(f"\n상세 결과가 'final_evaluation_results.csv'에 저장되었습니다.")
    
    print(f"\n=== 최종 평가 완료 ===")
    print("개선 과정: 기본 모델 → SFT 개선 → 디코딩 최적화")
    print("모든 단계가 성공적으로 완료되었습니다.")

if __name__ == "__main__":
    main()

KoChatGPT 최종 성능 평가를 시작합니다...
=== 최종 종합 성능 평가 ===
테스트 질문 수: 31
최적 디코딩 설정 적용 중...

진행률: 5/31
진행률: 10/31
진행률: 15/31
진행률: 20/31
진행률: 25/31
진행률: 30/31

최종 성능 분석 결과

=== 기본 성능 지표 ===
총 질문 수: 31
평균 응답 길이: 214.6자
평균 토큰 수: 48.9개
평균 생성 시간: 0.871초
평균 반복 비율: 0.000

=== 품질 지표 ===
완성도 높은 응답: 25/31 (80.6%)
적절한 길이 응답: 27/31 (87.1%)

=== 응답 길이 분포 ===
최단: 92자
최장: 330자
중간값: 223.0자
표준편차: 72.5자

=== 문제 응답 분석 ===
너무 짧은 응답 (50자 미만): 0개
너무 긴 응답 (300자 초과): 4개
반복이 많은 응답 (0.2 이상): 0개

=== 응답 품질 예시 ===

--- 우수 응답 예시 ---

질문: 인공지능이 뭐야?
응답: 저는 인공지능을 학습하는 AI입니다. 제가 알고 있는 정보로는, "AI는 인간의 감정을 이해하고 이해합니다.", token은 "그렇습니다." 라는 뜻일 수 있습니다.
길이: 92자, 반복도: 0.000

질문: 지구 온난화의 원인은?
응답: 지구의 온도는 상승하고 있으며, 지구온난화는 지구의 생물 다양성과 환경 오염에 기인합니다. 따라서 기후 변화에 대한 대책은 매우 중요하며, 이에 따라 다양한 대응책이 필요해집니다. 예를 들어, 식량, 물, 에너지 등 생활 필수 자원을 충분히 확보하고, 자연재해에 대비하는 것이 좋습니다. 또한, 대기오염을 줄이기 위해 적절한 조치를 취하거나, 화석연료 사용을 줄이는 등의 방법을 시도할 수 있습니다.
길이: 223자, 반복도: 0.000

질문: 감기 걸렸을 때 대처법
응답: 1. 손 씻기, 기침 예절 지키기\n2. 마스크 착용하기 - 감기에 걸린 경우 반드시 손을 깨끗하게 씻어주는 것이 좋습니다. 만약 증상이 심각하다

# RM 수동평가

In [58]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

# 평가할 질문들 (더 적은 수로)
test_questions = [
    {"question": "대한민국의 수도는?", "type": "factual"},
    {"question": "인공지능이 뭐야?", "type": "explanation"}, 
    {"question": "김치찌개 만드는 방법", "type": "how_to"},
    {"question": "스트레스 해소 방법은?", "type": "advice"},
    {"question": "파이썬 리스트와 튜플 차이점", "type": "explanation"},
    {"question": "감기 걸렸을 때 대처법", "type": "how_to"},
    {"question": "좋은 책 추천해줘", "type": "advice"}
]

def load_models():
    """모델들 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    try:
        model_old = AutoModelForCausalLM.from_pretrained("models/output_1_SFT")
        model_new = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
        print("두 모델 모두 로드 성공")
    except:
        print("모델 로드 실패")
        return None, None, None
    
    if torch.cuda.is_available():
        model_old = model_old.cuda()
        model_new = model_new.cuda()
    
    return model_old, model_new, tokenizer

def generate_response(model, tokenizer, question):
    """응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.4,
            top_k=25,
            top_p=0.8,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response.replace("'", "").strip()

def generate_comparison_data():
    """비교 데이터 생성 (평가는 따로)"""
    model_old, model_new, tokenizer = load_models()
    
    if model_old is None:
        return None
    
    results = []
    
    print("응답 생성 중...")
    
    for i, question_data in enumerate(test_questions):
        question = question_data["question"]
        
        # 두 모델 응답 생성
        response_old = generate_response(model_old, tokenizer, question)
        response_new = generate_response(model_new, tokenizer, question)
        
        result = {
            "질문번호": i+1,
            "질문": question,
            "유형": question_data["type"],
            "기존모델_응답": response_old,
            "개선모델_응답": response_new,
            "기존모델_길이": len(response_old),
            "개선모델_길이": len(response_new)
        }
        
        results.append(result)
        print(f"질문 {i+1}/{len(test_questions)} 완료")
    
    return results

def display_for_evaluation(results):
    """평가용 출력 (보기 좋게)"""
    print("\n" + "="*80)
    print("응답 비교 평가 시트")
    print("="*80)
    print("각 질문의 두 응답을 비교하여 어느 것이 더 좋은지 판단하세요")
    print("A=기존모델, B=개선모델, C=비슷함")
    print("="*80)
    
    for result in results:
        print(f"\n【질문 {result['질문번호']}】 {result['질문']} (유형: {result['유형']})")
        print("-" * 60)
        
        print(f"[A: 기존 모델] ({result['기존모델_길이']}자)")
        print(f"{result['기존모델_응답']}")
        
        print(f"\n[B: 개선 모델] ({result['개선모델_길이']}자)")
        print(f"{result['개선모델_응답']}")
        
        print(f"\n평가:")
        print(f"□ 정확성: A / B / C")
        print(f"□ 유용성: A / B / C") 
        print(f"□ 적절한길이: A / B / C")
        print(f"□ 전체평가: A / B / C")
        print("=" * 80)

def create_evaluation_sheet(results):
    """평가 시트 CSV 생성"""
    eval_data = []
    
    for result in results:
        eval_data.append({
            "질문번호": result["질문번호"],
            "질문": result["질문"],
            "유형": result["유형"],
            "기존모델응답": result["기존모델_응답"],
            "개선모델응답": result["개선모델_응답"],
            "기존길이": result["기존모델_길이"],
            "개선길이": result["개선모델_길이"],
            "정확성평가": "",  # 비워둠 - 수동 입력
            "유용성평가": "",  # 비워둠 - 수동 입력
            "길이평가": "",   # 비워둠 - 수동 입력
            "전체평가": "",   # 비워둠 - 수동 입력
            "메모": ""       # 비워둠 - 수동 입력
        })
    
    df = pd.DataFrame(eval_data)
    df.to_csv("evaluation_sheet.csv", index=False, encoding='utf-8-sig')
    print(f"\n평가 시트가 'evaluation_sheet.csv'에 저장되었습니다.")
    print("파일을 열어서 평가 결과를 A/B/C로 입력하세요.")

def analyze_results_from_csv(filename="evaluation_sheet.csv"):
    """CSV에서 평가 결과 분석"""
    try:
        df = pd.read_csv(filename, encoding='utf-8-sig')
        
        print("\n" + "="*50)
        print("평가 결과 분석")
        print("="*50)
        
        categories = ["정확성평가", "유용성평가", "길이평가", "전체평가"]
        
        for cat in categories:
            if cat in df.columns:
                a_count = sum(df[cat] == 'A')
                b_count = sum(df[cat] == 'B') 
                c_count = sum(df[cat] == 'C')
                total = a_count + b_count + c_count
                
                if total > 0:
                    print(f"\n{cat}:")
                    print(f"  기존모델(A): {a_count}승 ({a_count/total*100:.1f}%)")
                    print(f"  개선모델(B): {b_count}승 ({b_count/total*100:.1f}%)")
                    print(f"  비슷함(C): {c_count}회 ({c_count/total*100:.1f}%)")
        
        # 유형별 분석
        if "전체평가" in df.columns:
            print(f"\n유형별 개선모델 승률:")
            for q_type in df["유형"].unique():
                type_df = df[df["유형"] == q_type]
                b_wins = sum(type_df["전체평가"] == 'B')
                total = len(type_df)
                print(f"  {q_type}: {b_wins}/{total} ({b_wins/total*100:.1f}%)")
                
    except FileNotFoundError:
        print("evaluation_sheet.csv 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"분석 중 오류: {e}")

def main():
    """메인 실행 함수"""
    print("노트북용 모델 평가를 시작합니다...")
    
    # 1단계: 응답 생성
    results = generate_comparison_data()
    
    if results is None:
        print("모델 로드 실패로 종료합니다.")
        return
    
    # 2단계: 화면에 출력
    display_for_evaluation(results)
    
    # 3단계: CSV 파일 생성
    create_evaluation_sheet(results)
    
    print(f"\n다음 단계:")
    print(f"1. 'evaluation_sheet.csv' 파일을 엑셀이나 구글시트로 열기")
    print(f"2. 각 응답을 읽고 평가 컬럼에 A/B/C 입력")
    print(f"3. 저장 후 analyze_results_from_csv() 함수 실행")

# 분석만 실행하고 싶을 때
def quick_analysis():
    """평가 완료 후 결과만 분석"""
    analyze_results_from_csv()

if __name__ == "__main__":
    main()

노트북용 모델 평가를 시작합니다...
두 모델 모두 로드 성공
응답 생성 중...
질문 1/7 완료
질문 2/7 완료
질문 3/7 완료
질문 4/7 완료
질문 5/7 완료
질문 6/7 완료
질문 7/7 완료

응답 비교 평가 시트
각 질문의 두 응답을 비교하여 어느 것이 더 좋은지 판단하세요
A=기존모델, B=개선모델, C=비슷함

【질문 1】 대한민국의 수도는? (유형: factual)
------------------------------------------------------------
[A: 기존 모델] (120자)
저는 인공지능 어시스턴트이기 때문에 대한민국의 수도를 알 수 없습니다. 하지만 대한민국 수도라는 용어는 여러 가지 의미로 사용될 수 있습니다. 예를 들어, "서울" 또는 "세종", "청주"라는 용어가 있을 것입니다.

[B: 개선 모델] (95자)
저는 인공지능 언어모델로써 답변을 생성하고 있습니다. 따라서 대한민국 수도에 대한 정보가 없습니다. 추가적인 정보를 제공해주시면 더 정확한 대답을 드릴 수 있을 것입니다.

평가:
□ 정확성: A / B / C
□ 유용성: A / B / C
□ 적절한길이: A / B / C
□ 전체평가: A / B / C

【질문 2】 인공지능이 뭐야? (유형: explanation)
------------------------------------------------------------
[A: 기존 모델] (114자)
저는 인공지능을 이해하고 이해하기 위해 노력하고 있습니다. 하지만 인간의 지능은 매우 다양하며, 인간처럼 생각하거나 경험하지 못하는 경우도 많습니다. 따라서 인간과 같은 생각을 하지 않는 것이 중요합니다.

[B: 개선 모델] (203자)
저는 인공지능을 학습하는 AI이기 때문에, 인간과 같은 감정을 느끼지 않습니다. 하지만 인간의 감정에는 다양한 요소가 존재합니다. 예를 들어, 인간들은 자신의 감정과 관련된 정보를 가지고 있으며, 이를 바탕으로 문제를 해결하거나 해결하는 능력을 갖출 수 

# 평가 결과

In [60]:
analyze_results_from_csv()


평가 결과 분석

정확성평가:
  기존모델(A): 1승 (14.3%)
  개선모델(B): 5승 (71.4%)
  비슷함(C): 1회 (14.3%)

유용성평가:
  기존모델(A): 2승 (28.6%)
  개선모델(B): 4승 (57.1%)
  비슷함(C): 1회 (14.3%)

길이평가:
  기존모델(A): 2승 (28.6%)
  개선모델(B): 3승 (42.9%)
  비슷함(C): 2회 (28.6%)

전체평가:
  기존모델(A): 1승 (14.3%)
  개선모델(B): 5승 (71.4%)
  비슷함(C): 1회 (14.3%)

유형별 개선모델 승률:
  factual: 1/1 (100.0%)
  explanation: 1/2 (50.0%)
  how_to: 2/2 (100.0%)
  advice: 1/2 (50.0%)


# RM 추가 평가 진행

In [61]:
# 확장된 평가 질문 (30개, 균등 분배)
test_questions = [
    # 사실 질문 (8개)
    {"question": "대한민국의 수도는?", "type": "factual"},
    {"question": "1더하기 1은?", "type": "factual"},
    {"question": "지구에서 가장 높은 산은?", "type": "factual"},
    {"question": "한국의 화폐 단위는?", "type": "factual"},
    {"question": "태양계에서 가장 큰 행성은?", "type": "factual"},
    {"question": "물의 끓는점은?", "type": "factual"},
    {"question": "한국의 국화는?", "type": "factual"},
    {"question": "광복절은 언제?", "type": "factual"},
    
    # 설명 질문 (8개)
    {"question": "인공지능이 뭐야?", "type": "explanation"},
    {"question": "파이썬 리스트와 튜플 차이점", "type": "explanation"},
    {"question": "지구 온난화란?", "type": "explanation"},
    {"question": "블록체인 기술이란?", "type": "explanation"},
    {"question": "민주주의란 무엇인가?", "type": "explanation"},
    {"question": "코로나19가 뭐야?", "type": "explanation"},
    {"question": "메타버스란?", "type": "explanation"},
    {"question": "비트코인이란?", "type": "explanation"},
    
    # 방법 질문 (7개)  
    {"question": "김치찌개 만드는 방법", "type": "how_to"},
    {"question": "감기 걸렸을 때 대처법", "type": "how_to"},
    {"question": "스마트폰 배터리 절약법", "type": "how_to"},
    {"question": "라면 맛있게 끓이는 법", "type": "how_to"},
    {"question": "컴퓨터 속도 빠르게 하는 법", "type": "how_to"},
    {"question": "영어 발음 연습 방법", "type": "how_to"},
    {"question": "숙면을 위한 방법", "type": "how_to"},
    
    # 조언 질문 (7개)
    {"question": "스트레스 해소 방법은?", "type": "advice"},
    {"question": "시간 관리 팁", "type": "advice"},
    {"question": "좋은 책 추천해줘", "type": "advice"},
    {"question": "건강한 식단 조언", "type": "advice"},
    {"question": "새해 목표 세우는 법", "type": "advice"},
    {"question": "돈 모으는 방법", "type": "advice"},
    {"question": "좋은 습관 만들기", "type": "advice"}
]

def load_models():
    """모델들 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    try:
        model_old = AutoModelForCausalLM.from_pretrained("models/output_1_SFT")
        model_new = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
        print("모델 로드 성공")
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        return None, None, None
    
    if torch.cuda.is_available():
        model_old = model_old.cuda()
        model_new = model_new.cuda()
    
    return model_old, model_new, tokenizer

def generate_response(model, tokenizer, question):
    """응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.4,
            top_k=25,
            top_p=0.8,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response.replace("'", "").strip()

def generate_randomized_evaluation():
    """무작위 순서로 평가 데이터 생성"""
    model_old, model_new, tokenizer = load_models()
    
    if model_old is None:
        return None
    
    # 질문 순서 무작위화
    shuffled_questions = test_questions.copy()
    random.shuffle(shuffled_questions)
    
    results = []
    
    print(f"30개 질문에 대한 응답 생성 중...")
    
    for i, question_data in enumerate(shuffled_questions):
        question = question_data["question"]
        
        # 두 모델 응답 생성
        response_old = generate_response(model_old, tokenizer, question)
        response_new = generate_response(model_new, tokenizer, question)
        
        # 모델 순서도 무작위화 (블라인드 테스트)
        if random.choice([True, False]):
            first_model = "모델_A"
            second_model = "모델_B"
            first_response = response_old
            second_response = response_new
            correct_answer = "A"  # A가 기존, B가 개선
        else:
            first_model = "모델_B"
            second_model = "모델_A"
            first_response = response_new
            second_response = response_old
            correct_answer = "B"  # B가 기존, A가 개선
        
        result = {
            "질문번호": i+1,
            "질문": question,
            "유형": question_data["type"],
            f"{first_model}_응답": first_response,
            f"{second_model}_응답": second_response,
            f"{first_model}_길이": len(first_response),
            f"{second_model}_길이": len(second_response),
            "실제_개선모델": correct_answer,  # 평가 후 확인용
            "정확성평가": "",
            "유용성평가": "",
            "길이평가": "",
            "전체평가": "",
            "메모": ""
        }
        
        results.append(result)
        
        if (i + 1) % 5 == 0:
            print(f"{i + 1}/30 완료")
    
    return results


def create_csv_sheet(results):
    """CSV 평가 시트도 생성"""
    eval_data = []
    
    for result in results:
        eval_data.append({
            "질문번호": result["질문번호"],
            "질문": result["질문"],
            "유형": result["유형"],
            "모델A응답": result["모델_A_응답"],
            "모델B응답": result["모델_B_응답"],
            "모델A길이": result["모델_A_길이"],
            "모델B길이": result["모델_B_길이"],
            "실제개선모델": result["실제_개선모델"],
            "정확성평가": "",
            "유용성평가": "",
            "길이평가": "",
            "전체평가": "",
            "메모": ""
        })
    
    df = pd.DataFrame(eval_data)
    df.to_csv("evaluation_30questions.csv", index=False, encoding='utf-8-sig')
    print("evaluation_30questions.csv 파일도 생성되었습니다.")

def analyze_detailed_results(filename="evaluation_30questions.csv"):
    """상세한 결과 분석"""
    try:
        df = pd.read_csv(filename, encoding='utf-8-sig')
        
        print("\n" + "="*60)
        print("30개 질문 평가 결과 분석")
        print("="*60)
        
        categories = ["정확성평가", "유용성평가", "길이평가", "전체평가"]
        
        # 전체 승률
        for cat in categories:
            if cat in df.columns:
                # 실제 개선 모델이 이긴 경우 계산
                improved_wins = 0
                old_wins = 0
                ties = 0
                
                for _, row in df.iterrows():
                    if pd.isna(row[cat]) or row[cat] == "":
                        continue
                        
                    real_improved = row["실제개선모델"]
                    choice = row[cat]
                    
                    if choice == real_improved:
                        improved_wins += 1
                    elif choice == "C":
                        ties += 1
                    else:
                        old_wins += 1
                
                total = improved_wins + old_wins + ties
                if total > 0:
                    print(f"\n{cat}:")
                    print(f"  개선모델 승리: {improved_wins}/{total} ({improved_wins/total*100:.1f}%)")
                    print(f"  기존모델 승리: {old_wins}/{total} ({old_wins/total*100:.1f}%)")
                    print(f"  무승부: {ties}/{total} ({ties/total*100:.1f}%)")
        
        # 유형별 분석
        if "전체평가" in df.columns:
            print(f"\n유형별 개선모델 승률:")
            for q_type in df["유형"].unique():
                type_df = df[df["유형"] == q_type]
                improved_wins = 0
                total_evaluated = 0
                
                for _, row in type_df.iterrows():
                    if pd.isna(row["전체평가"]) or row["전체평가"] == "":
                        continue
                    total_evaluated += 1
                    if row["전체평가"] == row["실제개선모델"]:
                        improved_wins += 1
                
                if total_evaluated > 0:
                    print(f"  {q_type}: {improved_wins}/{total_evaluated} ({improved_wins/total_evaluated*100:.1f}%)")
        
        # 통계적 유의성 간단 체크
        if "전체평가" in df.columns:
            valid_responses = df[df["전체평가"].isin(["A", "B"])].copy()
            if len(valid_responses) >= 20:
                improved_wins = sum(valid_responses["전체평가"] == valid_responses["실제개선모델"])
                total = len(valid_responses)
                win_rate = improved_wins / total
                
                print(f"\n통계적 신뢰성:")
                print(f"  총 유효 평가: {total}개")
                print(f"  개선모델 승률: {win_rate:.1%}")
                
                if win_rate > 0.7 and total >= 20:
                    print(f"  => 통계적으로 의미있는 개선 효과")
                elif win_rate > 0.6 and total >= 30:
                    print(f"  => 개선 효과 있음")
                elif win_rate < 0.4:
                    print(f"  => 오히려 성능 저하")
                else:
                    print(f"  => 개선 효과 불분명")
                    
    except FileNotFoundError:
        print("evaluation_30questions.csv 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"분석 중 오류: {e}")

def main():
    """메인 실행"""
    print("30개 질문으로 확장된 모델 평가를 시작합니다...")
    print("블라인드 테스트를 위해 모델 순서를 무작위화합니다.")
    
    # 시드 설정 (재현 가능하도록)
    random.seed(42)
    
    # 응답 생성
    results = generate_randomized_evaluation()
    
    if results is None:
        print("모델 로드 실패로 종료합니다.")
        return
    
    # HTML과 CSV 모두 생성
    create_evaluation_html(results)
    create_csv_sheet(results)
    
    print(f"\n파일 생성 완료:")
    print(f"2. evaluation_30questions.csv - 엑셀에서 평가")
    print(f"\n평가 완료 후 analyze_detailed_results() 실행하세요.")

if __name__ == "__main__":
    main()

30개 질문으로 확장된 모델 평가를 시작합니다...
블라인드 테스트를 위해 모델 순서를 무작위화합니다.
모델 로드 성공
30개 질문에 대한 응답 생성 중...
5/30 완료
10/30 완료
15/30 완료
20/30 완료
25/30 완료
30/30 완료
evaluation_form.html 파일이 생성되었습니다.
브라우저에서 열어서 평가하고, 결과를 CSV에 입력하세요.
evaluation_30questions.csv 파일도 생성되었습니다.

파일 생성 완료:
1. evaluation_form.html - 브라우저에서 보기 좋게 평가
2. evaluation_30questions.csv - 엑셀에서 평가

평가 완료 후 analyze_detailed_results() 실행하세요.


# 추가 평가 후 결과

In [63]:
analyze_detailed_results()


30개 질문 평가 결과 분석

정확성평가:
  개선모델 승리: 16/30 (53.3%)
  기존모델 승리: 12/30 (40.0%)
  무승부: 2/30 (6.7%)

유용성평가:
  개선모델 승리: 14/30 (46.7%)
  기존모델 승리: 13/30 (43.3%)
  무승부: 3/30 (10.0%)

길이평가:
  개선모델 승리: 15/30 (50.0%)
  기존모델 승리: 11/30 (36.7%)
  무승부: 4/30 (13.3%)

전체평가:
  개선모델 승리: 15/30 (50.0%)
  기존모델 승리: 11/30 (36.7%)
  무승부: 4/30 (13.3%)

유형별 개선모델 승률:
  how_to: 2/7 (28.6%)
  explanation: 5/8 (62.5%)
  advice: 5/7 (71.4%)
  factual: 3/8 (37.5%)

통계적 신뢰성:
  총 유효 평가: 26개
  개선모델 승률: 57.7%
  => 개선 효과 불분명


# RM 평가 결과 학습

In [67]:
class FixedRM(RewardModel):
    """저장 문제를 해결한 RM 클래스"""
    
    def __init__(self, sft_model_path, tokenizer=None):
        # SFT 모델 로드
        sft_model = AutoModelForCausalLM.from_pretrained(sft_model_path)
        
        # GPT2Model 추출
        gpt2_model = GPT2Model(sft_model.config)
        gpt2_model.load_state_dict(sft_model.transformer.state_dict())
        
        if tokenizer is not None:
            gpt2_model.resize_token_embeddings(len(tokenizer))
        
        # Value head
        value_head = nn.Linear(gpt2_model.config.n_embd, 1)
        
        super().__init__(gpt2_model, value_head)
        
        self.sft_model_path = sft_model_path
        self.config = sft_model.config

def prepare_rm_data_fixed():
    """RM 데이터 준비 (수정된 버전)"""
    
    with open('KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as f:
        list_data_dict = json.load(f)
    
    pairwise_data = []
    
    for item in list_data_dict:
        prompt = item['prompt']
        completions = [item['completion_0'], item['completion_1'], item['completion_2']]
        ranking = item['ranking']
        
        # 모든 쌍 생성
        for i in range(3):
            for j in range(i+1, 3):
                if ranking[i] < ranking[j]:
                    chosen = completions[i]
                    rejected = completions[j]
                else:
                    chosen = completions[j]
                    rejected = completions[i]
                
                pairwise_data.append({
                    'prompt': prompt,
                    'chosen': chosen,
                    'rejected': rejected
                })
    
    # 데이터 섞기
    random.seed(42)
    random.shuffle(pairwise_data)
    
    print(f"총 RM 학습 데이터: {len(pairwise_data)}개")
    
    # 더 많은 데이터 사용 (5000개)
    if len(pairwise_data) > 5000:
        pairwise_data = pairwise_data[:5000]
        print(f"5000개로 제한")
    
    return pairwise_data

def train_rm_fixed():
    """수정된 RM 학습"""
    
    print("=== RM 학습 재시작 ===")
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=512
    )
    
    # RM 모델 생성
    print("개선된 SFT 모델 기반으로 RM 생성 중...")
    
    with NaiveStrategy().model_init_context():
        rm_model = FixedRM("models/improved_sft_v2", tokenizer).cuda()
    
    # 데이터 준비
    all_data = prepare_rm_data_fixed()
    
    # 학습/평가 데이터 분할
    split_idx = int(len(all_data) * 0.9)
    train_data = all_data[:split_idx]
    eval_data = all_data[split_idx:]
    
    print(f"학습 데이터: {len(train_data)}개")
    print(f"평가 데이터: {len(eval_data)}개")
    
    # 데이터셋 생성
    train_dataset = RewardDataset(train_data, tokenizer, 512)
    eval_dataset = RewardDataset(eval_data, tokenizer, 512)
    
    # 트레이너 설정 (학습률 대폭 감소)
    trainer = RewardModelTrainer(
        model=rm_model,
        strategy=NaiveStrategy(),
        optim=torch.optim.Adam(rm_model.parameters(), lr=1e-6),  # 10배 감소
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        batch_size=4,
        max_epochs=3  # 에포크 증가
    )
    
    print("RM 학습 시작...")
    trainer.fit(use_lora=0)
    
    # 수동 저장
    print("모델 저장 중...")
    save_dir = "models/fixed_rm"
    os.makedirs(save_dir, exist_ok=True)
    
    # 저장할 데이터
    save_data = {
        'model_state_dict': rm_model.model.state_dict(),
        'value_head_state_dict': rm_model.value_head.state_dict(),
        'config_dict': rm_model.config.to_dict(),
        'sft_model_path': rm_model.sft_model_path
    }
    
    # 저장
    save_path = f"{save_dir}/reward_model.pt"
    torch.save(save_data, save_path)
    
    print(f"모델 저장 완료: {save_path}")
    print(f"저장 확인: {os.path.exists(save_path)}")
    
    return rm_model, tokenizer

def test_fixed_rm():
    """수정된 RM 모델 테스트"""
    
    print("\n=== RM 모델 테스트 ===")
    
    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=512
    )
    
    try:
        # 저장된 모델 로드
        checkpoint = torch.load('models/fixed_rm/reward_model.pt')
        
        # 모델 재구성
        from transformers import GPT2Config
        config = GPT2Config.from_dict(checkpoint['config_dict'])
        
        gpt2_model = GPT2Model(config)
        gpt2_model.resize_token_embeddings(len(tokenizer))
        
        value_head = nn.Linear(config.n_embd, 1)
        
        rm_model = RewardModel(gpt2_model, value_head)
        
        # 가중치 로드
        rm_model.model.load_state_dict(checkpoint['model_state_dict'])
        rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
        
        if torch.cuda.is_available():
            rm_model = rm_model.cuda()
        
        print("RM 모델 로드 성공!")
        
        # 다양한 품질의 텍스트로 테스트
        test_cases = [
            # 좋은 답변들
            "대한민국의 수도는 서울입니다.",
            "인공지능은 컴퓨터가 인간의 지능을 모방하여 학습하고 추론하는 기술입니다.",
            "김치찌개를 만들려면 신김치를 기름에 볶고 물을 넣어 끓인 후 두부와 고기를 넣으면 됩니다.",
            
            # 나쁜 답변들
            "몰라요.",
            "ㅋㅋㅋㅋ",
            "그런 거 왜 물어봐요?",
            
            # 중간 답변들
            "잘 모르겠지만 찾아보시면 좋을 것 같습니다.",
            "그 질문은 복잡해서 간단히 답하기 어렵습니다."
        ]
        
        print("\nRM 점수 테스트:")
        scores = []
        
        for text in test_cases:
            input_ids = tokenizer.encode(text, return_tensors='pt')
            if torch.cuda.is_available():
                input_ids = input_ids.cuda()
            
            with torch.no_grad():
                score = rm_model(input_ids).cpu().item()
            
            scores.append((score, text))
            print(f"{score:6.3f}: {text}")
        
        # 점수 순으로 정렬
        print(f"\n=== 점수 순 정렬 (높은 점수 = 더 좋은 답변) ===")
        scores.sort(reverse=True)
        
        for i, (score, text) in enumerate(scores):
            print(f"{i+1}위 ({score:6.3f}): {text}")
        
        print(f"\nRM 모델이 제대로 작동하는지 확인:")
        print(f"- 좋은 답변이 높은 점수를 받는가?")
        print(f"- 나쁜 답변이 낮은 점수를 받는가?")
        
        return rm_model, tokenizer
        
    except Exception as e:
        print(f"RM 모델 테스트 실패: {e}")
        return None, None

def main():
    print("RM 학습을 다시 진행합니다.")
    print("예상 소요 시간: 30-40분")
    
    # RM 학습
    rm_model, tokenizer = train_rm_fixed()
    
    if rm_model is not None:
        print("\n학습 완료! 테스트 진행...")
        test_fixed_rm()
        
        print(f"\n=== 최종 결과 ===")
        print(f"✓ RM 모델 학습 완료")
        print(f"✓ models/fixed_rm/reward_model.pt 저장 완료")
        print(f"✓ 다음 단계: 3-way 모델 비교 (기존SFT vs 개선SFT vs SFT+RM)")
    
    else:
        print("RM 학습 실패")

if __name__ == "__main__":
    main()

RM 학습을 다시 진행합니다.
예상 소요 시간: 30-40분
=== RM 학습 재시작 ===
개선된 SFT 모델 기반으로 RM 생성 중...
총 RM 학습 데이터: 30660개
5000개로 제한
학습 데이터: 4500개
평가 데이터: 500개


  0%|          | 0/4500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

RM 학습 시작...


Train epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train step of epoch 0:   0%|          | 0/1125 [00:00<?, ?it/s]

Train step of epoch 1:   0%|          | 0/1125 [00:00<?, ?it/s]

Train step of epoch 2:   0%|          | 0/1125 [00:00<?, ?it/s]

모델 저장 중...
모델 저장 완료: models/fixed_rm/reward_model.pt
저장 확인: True

학습 완료! 테스트 진행...

=== RM 모델 테스트 ===
RM 모델 로드 성공!

RM 점수 테스트:
-0.042: 대한민국의 수도는 서울입니다.
 1.307: 인공지능은 컴퓨터가 인간의 지능을 모방하여 학습하고 추론하는 기술입니다.
 1.121: 김치찌개를 만들려면 신김치를 기름에 볶고 물을 넣어 끓인 후 두부와 고기를 넣으면 됩니다.
-1.097: 몰라요.
-1.659: ㅋㅋㅋㅋ
-0.541: 그런 거 왜 물어봐요?
 0.211: 잘 모르겠지만 찾아보시면 좋을 것 같습니다.
-0.201: 그 질문은 복잡해서 간단히 답하기 어렵습니다.

=== 점수 순 정렬 (높은 점수 = 더 좋은 답변) ===
1위 ( 1.307): 인공지능은 컴퓨터가 인간의 지능을 모방하여 학습하고 추론하는 기술입니다.
2위 ( 1.121): 김치찌개를 만들려면 신김치를 기름에 볶고 물을 넣어 끓인 후 두부와 고기를 넣으면 됩니다.
3위 ( 0.211): 잘 모르겠지만 찾아보시면 좋을 것 같습니다.
4위 (-0.042): 대한민국의 수도는 서울입니다.
5위 (-0.201): 그 질문은 복잡해서 간단히 답하기 어렵습니다.
6위 (-0.541): 그런 거 왜 물어봐요?
7위 (-1.097): 몰라요.
8위 (-1.659): ㅋㅋㅋㅋ

RM 모델이 제대로 작동하는지 확인:
- 좋은 답변이 높은 점수를 받는가?
- 나쁜 답변이 낮은 점수를 받는가?

=== 최종 결과 ===
✓ RM 모델 학습 완료
✓ models/fixed_rm/reward_model.pt 저장 완료
✓ 다음 단계: 3-way 모델 비교 (기존SFT vs 개선SFT vs SFT+RM)


In [65]:
import os
print("RM 모델 저장 확인:", os.path.exists("models/improved_rm/reward_model.pt"))

RM 모델 저장 확인: False


In [68]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from chatgpt.models.base import RewardModel
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import pandas as pd
import random

def load_all_three_models():
    """세 단계 모델 모두 로드"""
    
    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    models = {}
    
    # 1. 기존 SFT 모델
    try:
        models['기존SFT'] = AutoModelForCausalLM.from_pretrained("models/output_1_SFT")
        print("✓ 기존 SFT 모델 로드 성공")
    except Exception as e:
        print(f"✗ 기존 SFT 모델 로드 실패: {e}")
        return None
    
    # 2. 개선 SFT 모델
    try:
        models['개선SFT'] = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
        print("✓ 개선 SFT 모델 로드 성공")
    except Exception as e:
        print(f"✗ 개선 SFT 모델 로드 실패: {e}")
        return None
    
    # 3. RM 모델
    try:
        from transformers import GPT2Config
        checkpoint = torch.load('models/fixed_rm/reward_model.pt')
        
        # RM 모델 재구성
        config = GPT2Config.from_dict(checkpoint['config_dict'])
        gpt2_model = GPT2Model(config)
        gpt2_model.resize_token_embeddings(len(tokenizer))
        
        value_head = nn.Linear(config.n_embd, 1)
        rm_model = RewardModel(gpt2_model, value_head)
        
        # 가중치 로드
        rm_model.model.load_state_dict(checkpoint['model_state_dict'])
        rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
        
        models['RM'] = rm_model
        print("✓ RM 모델 로드 성공")
        
    except Exception as e:
        print(f"✗ RM 모델 로드 실패: {e}")
        models['RM'] = None
    
    # GPU로 이동
    if torch.cuda.is_available():
        for name, model in models.items():
            if model is not None:
                models[name] = model.cuda()
    
    return models, tokenizer

def generate_with_rm_selection(sft_model, rm_model, tokenizer, question, num_candidates=3):
    """RM 점수를 이용한 베스트 응답 선택"""
    
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    candidates = []
    
    # 여러 후보 응답 생성
    for _ in range(num_candidates):
        with torch.no_grad():
            outputs = sft_model.generate(
                inputs,
                max_length=inputs.shape[1] + 120,
                do_sample=True,
                temperature=0.7,  # 다양성을 위해 조금 높게
                top_k=50,
                top_p=0.9,
                repetition_penalty=1.2,
                pad_token_id=tokenizer.pad_token_id
            )
        
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        response = response.replace("'", "").strip()
        candidates.append(response)
    
    # RM으로 각 후보의 점수 계산
    best_response = ""
    best_score = float('-inf')
    
    for candidate in candidates:
        # RM 입력용 텍스트 (질문 + 응답)
        full_text = prompt + candidate
        rm_inputs = tokenizer.encode(full_text, return_tensors='pt', truncation=True, max_length=512)
        
        if torch.cuda.is_available():
            rm_inputs = rm_inputs.cuda()
        
        with torch.no_grad():
            score = rm_model(rm_inputs).cpu().item()
        
        if score > best_score:
            best_score = score
            best_response = candidate
    
    return best_response, best_score

def generate_standard_response(model, tokenizer, question):
    """표준 응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.4,
            top_k=25,
            top_p=0.8,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response.replace("'", "").strip()

def run_three_way_comparison():
    """3단계 모델 비교 실험"""
    
    models, tokenizer = load_all_three_models()
    
    if models is None or models['RM'] is None:
        print("모델 로드 실패")
        return None
    
    # 비교용 질문들 (다양한 유형)
    test_questions = [
        # 사실 질문
        {"question": "대한민국의 수도는?", "type": "factual"},
        {"question": "지구에서 가장 높은 산은?", "type": "factual"},
        {"question": "1더하기 1은?", "type": "factual"},
        
        # 설명 질문  
        {"question": "인공지능이 뭐야?", "type": "explanation"},
        {"question": "지구 온난화란?", "type": "explanation"},
        {"question": "블록체인 기술이란?", "type": "explanation"},
        
        # 방법 질문
        {"question": "김치찌개 만드는 방법", "type": "how_to"},
        {"question": "감기 걸렸을 때 대처법", "type": "how_to"},
        {"question": "영어 공부 방법", "type": "how_to"},
        
        # 조언 질문
        {"question": "스트레스 해소 방법은?", "type": "advice"},
        {"question": "시간 관리 팁", "type": "advice"},
        {"question": "좋은 책 추천해줘", "type": "advice"}
    ]
    
    results = []
    
    print("3단계 모델 비교 실험 시작...")
    print(f"총 {len(test_questions)}개 질문 테스트")
    
    for i, q_data in enumerate(test_questions):
        question = q_data["question"]
        q_type = q_data["type"]
        
        print(f"\n진행률: {i+1}/{len(test_questions)} - {question}")
        
        # 1. 기존 SFT 응답
        response_old = generate_standard_response(models['기존SFT'], tokenizer, question)
        
        # 2. 개선 SFT 응답  
        response_new = generate_standard_response(models['개선SFT'], tokenizer, question)
        
        # 3. SFT + RM 응답 (개선 SFT + RM 선택)
        response_rm, rm_score = generate_with_rm_selection(
            models['개선SFT'], models['RM'], tokenizer, question
        )
        
        result = {
            "질문번호": i+1,
            "질문": question,
            "유형": q_type,
            "기존SFT_응답": response_old,
            "개선SFT_응답": response_new,
            "SFT_RM_응답": response_rm,
            "RM_점수": rm_score,
            "기존SFT_길이": len(response_old),
            "개선SFT_길이": len(response_new),
            "SFT_RM_길이": len(response_rm)
        }
        
        results.append(result)
    
    return results

def create_three_way_evaluation_sheet(results):
    """3단계 비교 평가 시트 생성"""
    
    eval_data = []
    
    for result in results:
        # 모델 순서 랜덤화 (블라인드 테스트)
        models = ["기존SFT", "개선SFT", "SFT+RM"]
        responses = [
            result["기존SFT_응답"],
            result["개선SFT_응답"], 
            result["SFT_RM_응답"]
        ]
        lengths = [
            result["기존SFT_길이"],
            result["개선SFT_길이"],
            result["SFT_RM_길이"]
        ]
        
        # 무작위 섞기
        combined = list(zip(models, responses, lengths))
        random.shuffle(combined)
        
        shuffled_models, shuffled_responses, shuffled_lengths = zip(*combined)
        
        eval_data.append({
            "질문번호": result["질문번호"],
            "질문": result["질문"],
            "유형": result["유형"],
            "모델A": shuffled_models[0],
            "모델A_응답": shuffled_responses[0],
            "모델A_길이": shuffled_lengths[0],
            "모델B": shuffled_models[1], 
            "모델B_응답": shuffled_responses[1],
            "모델B_길이": shuffled_lengths[1],
            "모델C": shuffled_models[2],
            "모델C_응답": shuffled_responses[2], 
            "모델C_길이": shuffled_lengths[2],
            "RM_점수": result["RM_점수"],
            "최고_응답": "",  # A, B, C 중 선택
            "최악_응답": "",  # A, B, C 중 선택
            "정확성_순위": "",  # A>B>C 형식
            "유용성_순위": "",  # A>B>C 형식  
            "메모": ""
        })
    
    df = pd.DataFrame(eval_data)
    df.to_csv("three_way_comparison.csv", index=False, encoding='utf-8-sig')
    
    print(f"\n3단계 비교 평가 시트가 'three_way_comparison.csv'에 저장되었습니다.")
    print("각 질문에 대해 3개 모델의 응답을 비교하여 평가하세요:")
    print("- 최고_응답: A, B, C 중 가장 좋은 응답")
    print("- 최악_응답: A, B, C 중 가장 나쁜 응답") 
    print("- 정확성_순위: A>B>C 또는 C>A>B 형식으로 순위")

def display_comparison_preview(results, num_examples=3):
    """비교 결과 미리보기"""
    
    print(f"\n=== 3단계 모델 비교 미리보기 ===")
    
    for i in range(min(num_examples, len(results))):
        result = results[i]
        
        print(f"\n【질문 {result['질문번호']}】 {result['질문']} (유형: {result['유형']})")
        print("-" * 80)
        
        print(f"[기존 SFT] ({result['기존SFT_길이']}자)")
        print(f"{result['기존SFT_응답']}")
        
        print(f"\n[개선 SFT] ({result['개선SFT_길이']}자)")
        print(f"{result['개선SFT_응답']}")
        
        print(f"\n[SFT + RM] ({result['SFT_RM_길이']}자, RM점수: {result['RM_점수']:.3f})")
        print(f"{result['SFT_RM_응답']}")
        print("=" * 80)

def main():
    """메인 실행"""
    print("=== 3단계 RLHF 모델 비교 ===")
    print("Base → SFT → SFT+RM 단계별 성능 비교")
    
    # 시드 설정
    random.seed(42)
    
    # 비교 실험 실행
    results = run_three_way_comparison()
    
    if results:
        # 미리보기 출력
        display_comparison_preview(results)
        
        # 평가 시트 생성
        create_three_way_evaluation_sheet(results)
        
        print(f"\n=== 실험 완료 ===")
        print(f"- 총 {len(results)}개 질문 테스트")
        print(f"- 각 질문에 대해 3가지 방법으로 응답 생성")
        print(f"- CSV 파일에서 수동 평가 후 결과 분석 가능")
        
    else:
        print("실험 실패")

if __name__ == "__main__":
    main()

=== 3단계 RLHF 모델 비교 ===
Base → SFT → SFT+RM 단계별 성능 비교
✓ 기존 SFT 모델 로드 성공
✓ 개선 SFT 모델 로드 성공
✓ RM 모델 로드 성공
3단계 모델 비교 실험 시작...
총 12개 질문 테스트

진행률: 1/12 - 대한민국의 수도는?

진행률: 2/12 - 지구에서 가장 높은 산은?

진행률: 3/12 - 1더하기 1은?

진행률: 4/12 - 인공지능이 뭐야?

진행률: 5/12 - 지구 온난화란?

진행률: 6/12 - 블록체인 기술이란?

진행률: 7/12 - 김치찌개 만드는 방법

진행률: 8/12 - 감기 걸렸을 때 대처법

진행률: 9/12 - 영어 공부 방법

진행률: 10/12 - 스트레스 해소 방법은?

진행률: 11/12 - 시간 관리 팁

진행률: 12/12 - 좋은 책 추천해줘

=== 3단계 모델 비교 미리보기 ===

【질문 1】 대한민국의 수도는? (유형: factual)
--------------------------------------------------------------------------------
[기존 SFT] (99자)
저는 인공지능 어시스턴트이기 때문에 정확한 답변을 제공할 수 없습니다. 하지만 대한민국 수도에 대한 정보는 해당 지역의 부동산 중개업체나 관련 기관에 문의하시는 것이 좋겠습니다!

[개선 SFT] (78자)
저는 인공지능 언어모델로써 답변을 제공합니다.\n\n대한민국 수도는 대한민국의 수도이며, 대한민국 서울특별시 중구 을지로에 위치해 있습니다.

[SFT + RM] (146자, RM점수: 0.903)
저는 인공지능 언어모델이며, 대한민국의 수도는 대한민국입니다. 따라서 대한민국의 수도를 정확히 알 수 없습니다. 하지만 일반적으로 수도권은 서울특별시, 경기도와 함께 대한민국의 대표적인 도시 중 하나로, 서울과 경기 지역을 중심으로 발전해왔다고 알려져 있습니다.

【질문 2】 지구에서 가장 높은 산은? (유형: factual)
------------

In [1]:
import pandas as pd

def analyze_three_way_results(filename="three_way_comparison.csv"):
    """3-way 비교 결과 간단 분석"""
    
    try:
        df = pd.read_csv(filename, encoding='utf-8-sig')
        
        print("\n" + "="*50)
        print("3단계 모델 비교 결과 분석")
        print("="*50)
        
        # 최고 응답 분석
        if "최고_응답" in df.columns:
            model_wins = {"기존SFT": 0, "개선SFT": 0, "SFT+RM": 0}
            total = 0
            
            for _, row in df.iterrows():
                if pd.isna(row["최고_응답"]) or row["최고_응답"] == "":
                    continue
                    
                choice = row["최고_응답"]
                if choice == "A":
                    winner = row["모델A"]
                elif choice == "B":
                    winner = row["모델B"]
                elif choice == "C":
                    winner = row["모델C"]
                else:
                    continue
                
                if winner in model_wins:
                    model_wins[winner] += 1
                    total += 1
            
            print(f"\n최고 응답 승률:")
            for model, wins in model_wins.items():
                percentage = (wins/total*100) if total > 0 else 0
                print(f"  {model}: {wins}/{total} ({percentage:.1f}%)")
        
        # 유형별 분석
        if "최고_응답" in df.columns and "유형" in df.columns:
            print(f"\n유형별 최고 응답:")
            for q_type in df["유형"].unique():
                type_df = df[df["유형"] == q_type]
                print(f"\n  {q_type} 유형:")
                
                type_wins = {"기존SFT": 0, "개선SFT": 0, "SFT+RM": 0}
                type_total = 0
                
                for _, row in type_df.iterrows():
                    if pd.isna(row["최고_응답"]) or row["최고_응답"] == "":
                        continue
                        
                    choice = row["최고_응답"]
                    if choice == "A":
                        winner = row["모델A"]
                    elif choice == "B":
                        winner = row["모델B"]
                    elif choice == "C":
                        winner = row["모델C"]
                    else:
                        continue
                    
                    if winner in type_wins:
                        type_wins[winner] += 1
                        type_total += 1
                
                for model, wins in type_wins.items():
                    percentage = (wins/type_total*100) if type_total > 0 else 0
                    print(f"    {model}: {wins}/{type_total} ({percentage:.1f}%)")
        
        # 통계적 의미
        if total >= 10:
            best_model = max(model_wins, key=model_wins.get)
            best_rate = model_wins[best_model] / total
            
            print(f"\n결론:")
            if best_rate > 0.5:
                print(f"  {best_model}가 {best_rate:.1%} 승률로 최고 성능")
            else:
                print(f"  모델 간 성능 차이 미미 (최고: {best_model} {best_rate:.1%})")
                
    except FileNotFoundError:
        print("three_way_comparison.csv 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"분석 오류: {e}")

# 실행
if __name__ == "__main__":
    analyze_three_way_results()


3단계 모델 비교 결과 분석

최고 응답 승률:
  기존SFT: 3/12 (25.0%)
  개선SFT: 2/12 (16.7%)
  SFT+RM: 7/12 (58.3%)

유형별 최고 응답:

  factual 유형:
    기존SFT: 1/3 (33.3%)
    개선SFT: 0/3 (0.0%)
    SFT+RM: 2/3 (66.7%)

  explanation 유형:
    기존SFT: 1/3 (33.3%)
    개선SFT: 0/3 (0.0%)
    SFT+RM: 2/3 (66.7%)

  how_to 유형:
    기존SFT: 1/3 (33.3%)
    개선SFT: 1/3 (33.3%)
    SFT+RM: 1/3 (33.3%)

  advice 유형:
    기존SFT: 0/3 (0.0%)
    개선SFT: 1/3 (33.3%)
    SFT+RM: 2/3 (66.7%)

결론:
  SFT+RM가 58.3% 승률로 최고 성능


# PPO 진행

In [4]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.base import RewardModel
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from copy import deepcopy
import json
import random
import pandas as pd

def load_models_for_ppo():
    """PPO 학습을 위한 모델들 로드"""
    
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=512
    )
    
    print("PPO 학습용 모델들 로드 중...")
    
    # 1. Actor (개선된 SFT 모델 기반)
    with NaiveStrategy().model_init_context():
        actor = GPTActor(pretrained='models/improved_sft_v2', lora_rank=0)
        print("✓ Actor 모델 로드")
    
    # 2. Critic (RM 모델과 동일한 구조)
    try:
        checkpoint = torch.load('models/fixed_rm/reward_model.pt')
        
        with NaiveStrategy().model_init_context():
            # Critic은 RM과 동일한 구조 사용
            from transformers import GPT2Config
            config = GPT2Config.from_dict(checkpoint['config_dict'])
            
            critic = GPTCritic(pretrained='models/improved_sft_v2', lora_rank=0)
            print("✓ Critic 모델 로드")
            
    except Exception as e:
        print(f"Critic 로드 실패: {e}")
        return None
    
    # 3. Reward Model (이미 학습된 것)
    try:
        gpt2_model = GPT2Model(config)
        gpt2_model.resize_token_embeddings(len(tokenizer))
        value_head = nn.Linear(config.n_embd, 1)
        reward_model = RewardModel(gpt2_model, value_head)
        
        reward_model.model.load_state_dict(checkpoint['model_state_dict'])
        reward_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
        print("✓ Reward 모델 로드")
        
    except Exception as e:
        print(f"Reward 모델 로드 실패: {e}")
        return None
    
    # 4. Initial Model (참조용)
    initial_model = deepcopy(actor)
    print("✓ Initial 모델 생성")
    
    return actor, critic, reward_model, initial_model, tokenizer

def prepare_ppo_data():
    """PPO 학습용 프롬프트 데이터 준비"""
    
    with open('KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as f:
        list_data_dict = json.load(f)
    
    prompts = [item['prompt'] for item in list_data_dict]
    
    # 빠른 학습을 위해 일부만 사용
    random.seed(42)
    if len(prompts) > 1000:
        prompts = random.sample(prompts, 1000)
    
    print(f"PPO 학습용 프롬프트: {len(prompts)}개")
    return prompts

def tokenize_fn_ppo(texts, tokenizer):
    """PPO용 토크나이징 함수"""
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() if torch.cuda.is_available() else v for k, v in batch.items()}

def train_ppo_model():
    """PPO 모델 학습"""
    
    print("=== PPO 학습 시작 ===")
    
    # 모델들 로드
    models_result = load_models_for_ppo()
    if models_result is None:
        print("PPO 모델 로드 실패")
        return None
    
    actor, critic, reward_model, initial_model, tokenizer = models_result
    
    # GPU로 이동
    if torch.cuda.is_available():
        actor = actor.cuda()
        critic = critic.cuda()
        reward_model = reward_model.cuda()
        initial_model = initial_model.cuda()
    
    # 옵티마이저
    actor_optim = torch.optim.Adam(actor.parameters(), lr=1e-6)  # 매우 작은 학습률
    critic_optim = torch.optim.Adam(critic.parameters(), lr=1e-6)
    
    # Strategy로 준비
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
        (actor, actor_optim), (critic, critic_optim), reward_model, initial_model
    )
    
    # 프롬프트 데이터
    prompts = prepare_ppo_data()
    
    # PPO 트레이너
    trainer = PPOTrainer(
        strategy=NaiveStrategy(),
        actor=actor,
        critic=critic,
        reward_model=reward_model,
        initial_model=initial_model,
        actor_optim=actor_optim,
        critic_optim=critic_optim,
        max_epochs=1,  # 빠른 테스트용
        train_batch_size=8,
        tokenizer=lambda texts: tokenize_fn_ppo(texts, tokenizer),
        max_length=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    
    print("PPO 학습 시작...")
    print("예상 소요시간: 20-30분")
    
    try:
        trainer.fit(
            prompts,
            num_episodes=5,  # 에피소드 수 줄임
            max_timesteps=3,
            update_timesteps=3
        )
        
        # 모델 저장
        actor.model.save_pretrained('models/ppo_final')
        tokenizer.save_pretrained('models/ppo_final')
        
        print("PPO 학습 완료!")
        return actor, tokenizer
        
    except Exception as e:
        print(f"PPO 학습 실패: {e}")
        return None, None

def run_four_way_comparison():
    """4단계 완전 비교"""
    
    print("=== 4단계 RLHF 완전 비교 ===")
    
    # 토크나이저
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    # 모델들 로드
    models = {}
    
    try:
        # 1. 기존 SFT
        models['기존SFT'] = AutoModelForCausalLM.from_pretrained("models/output_1_SFT")
        
        # 2. 개선 SFT  
        models['개선SFT'] = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
        
        # 3. PPO 모델
        models['PPO'] = AutoModelForCausalLM.from_pretrained("models/ppo_final")
        
        print("4개 모델 모두 로드 성공")
        
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        return None
    
    # GPU로 이동
    if torch.cuda.is_available():
        for name, model in models.items():
            models[name] = model.cuda()
    
    # RM 모델도 로드 (SFT+RM용)
    try:
        from transformers import GPT2Config
        checkpoint = torch.load('models/fixed_rm/reward_model.pt')
        config = GPT2Config.from_dict(checkpoint['config_dict'])
        
        gpt2_model = GPT2Model(config)
        gpt2_model.resize_token_embeddings(len(tokenizer))
        value_head = nn.Linear(config.n_embd, 1)
        rm_model = RewardModel(gpt2_model, value_head)
        
        rm_model.model.load_state_dict(checkpoint['model_state_dict'])
        rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
        
        if torch.cuda.is_available():
            rm_model = rm_model.cuda()
            
        print("RM 모델도 로드 성공")
        
    except Exception as e:
        print(f"RM 모델 로드 실패: {e}")
        rm_model = None
    
    # 테스트 질문들
    test_questions = [
        {"question": "인공지능이 뭐야?", "type": "explanation"},
        {"question": "김치찌개 만드는 방법", "type": "how_to"},
        {"question": "스트레스 해소 방법은?", "type": "advice"},
        {"question": "대한민국의 수도는?", "type": "factual"},
        {"question": "파이썬 리스트와 튜플 차이점", "type": "explanation"},
        {"question": "영어 공부 방법", "type": "how_to"},
        {"question": "시간 관리 팁", "type": "advice"},
        {"question": "지구에서 가장 높은 산은?", "type": "factual"}
    ]
    
    results = []
    
    print(f"4단계 비교 테스트 시작... ({len(test_questions)}개 질문)")
    
    for i, q_data in enumerate(test_questions):
        question = q_data["question"]
        q_type = q_data["type"]
        
        print(f"진행률: {i+1}/{len(test_questions)} - {question}")
        
        # 각 모델로 응답 생성
        responses = {}
        
        for model_name, model in models.items():
            prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
            inputs = tokenizer.encode(prompt, return_tensors='pt')
            
            if torch.cuda.is_available():
                inputs = inputs.cuda()
            
            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 100,
                    do_sample=True,
                    temperature=0.4,
                    top_k=25,
                    top_p=0.8,
                    repetition_penalty=1.2,
                    pad_token_id=tokenizer.pad_token_id
                )
            
            response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
            response = response.replace("'", "").strip()
            responses[model_name] = response
        
        # SFT+RM 응답 (개선SFT + RM 선택)
        if rm_model is not None:
            sft_rm_response, rm_score = generate_with_rm_selection_simple(
                models['개선SFT'], rm_model, tokenizer, question
            )
            responses['SFT+RM'] = sft_rm_response
        
        result = {
            "질문번호": i+1,
            "질문": question,
            "유형": q_type,
            **{f"{name}_응답": resp for name, resp in responses.items()},
            **{f"{name}_길이": len(resp) for name, resp in responses.items()}
        }
        
        results.append(result)
    
    return results

def generate_with_rm_selection_simple(sft_model, rm_model, tokenizer, question):
    """간단한 RM 선택 함수"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    candidates = []
    for _ in range(3):
        with torch.no_grad():
            outputs = sft_model.generate(
                inputs,
                max_length=inputs.shape[1] + 100,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id
            )
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        candidates.append(response.replace("'", "").strip())
    
    best_response = ""
    best_score = float('-inf')
    
    for candidate in candidates:
        full_text = prompt + candidate
        rm_inputs = tokenizer.encode(full_text, return_tensors='pt', truncation=True, max_length=512)
        if torch.cuda.is_available():
            rm_inputs = rm_inputs.cuda()
        
        with torch.no_grad():
            score = rm_model(rm_inputs).cpu().item()
        
        if score > best_score:
            best_score = score
            best_response = candidate
    
    return best_response, best_score

def create_four_way_evaluation_sheet(results):
    """4단계 비교 평가 시트 생성"""
    
    eval_data = []
    
    for result in results:
        # 4개 모델 응답 랜덤 배치
        models = list(result.keys())
        response_models = [k for k in models if k.endswith('_응답')]
        model_names = [k.replace('_응답', '') for k in response_models]
        
        responses = [result[k] for k in response_models]
        lengths = [result[k.replace('_응답', '_길이')] for k in response_models]
        
        # 랜덤 섞기
        combined = list(zip(model_names, responses, lengths))
        random.shuffle(combined)
        shuffled_models, shuffled_responses, shuffled_lengths = zip(*combined)
        
        eval_data.append({
            "질문번호": result["질문번호"],
            "질문": result["질문"],
            "유형": result["유형"],
            **{f"모델{chr(65+i)}": shuffled_models[i] for i in range(len(shuffled_models))},
            **{f"모델{chr(65+i)}_응답": shuffled_responses[i] for i in range(len(shuffled_responses))},
            **{f"모델{chr(65+i)}_길이": shuffled_lengths[i] for i in range(len(shuffled_lengths))},
            "최고_모델": "",  # A, B, C, D 중 선택
            "최악_모델": "",  # A, B, C, D 중 선택
            "순위": "",      # A>B>C>D 형식
            "메모": ""
        })
    
    df = pd.DataFrame(eval_data)
    df.to_csv("four_way_comparison.csv", index=False, encoding='utf-8-sig')
    
    print(f"4단계 비교 평가 시트가 'four_way_comparison.csv'에 저장되었습니다.")

def main():
    """메인 실행"""
    print("완전한 RLHF 파이프라인 실험을 시작합니다.")
    print("단계: Base → SFT → SFT+RM → PPO")
    
    # PPO 학습
    ppo_model, tokenizer = train_ppo_model()
    
    if ppo_model is not None:
        print("\nPPO 학습 완료! 4단계 비교 시작...")
        
        # 4단계 비교
        results = run_four_way_comparison()
        
        if results:
            create_four_way_evaluation_sheet(results)
            print(f"\n=== 실험 완료 ===")
            print(f"완전한 RLHF 파이프라인 구축 및 비교 완료")
        else:
            print("4단계 비교 실패")
    else:
        print("PPO 학습 실패로 실험 중단")

if __name__ == "__main__":
    main()

완전한 RLHF 파이프라인 실험을 시작합니다.
단계: Base → SFT → SFT+RM → PPO
=== PPO 학습 시작 ===


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

PPO 학습용 모델들 로드 중...
✓ Actor 모델 로드
✓ Critic 모델 로드
✓ Reward 모델 로드
✓ Initial 모델 생성
PPO 학습용 프롬프트: 1000개
PPO 학습 시작...
예상 소요시간: 20-30분


Episode [1/5]:   0%|          | 0/3 [00:00<?, ?it/s]

...

Episode [5/5]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/1]:   0%|          | 0/3 [00:00<?, ?it/s]

PPO 학습 완료!

PPO 학습 완료! 4단계 비교 시작...
=== 4단계 RLHF 완전 비교 ===
4개 모델 모두 로드 성공


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


RM 모델도 로드 성공
4단계 비교 테스트 시작... (8개 질문)
진행률: 1/8 - 인공지능이 뭐야?
진행률: 2/8 - 김치찌개 만드는 방법
진행률: 3/8 - 스트레스 해소 방법은?
진행률: 4/8 - 대한민국의 수도는?
진행률: 5/8 - 파이썬 리스트와 튜플 차이점
진행률: 6/8 - 영어 공부 방법
진행률: 7/8 - 시간 관리 팁
진행률: 8/8 - 지구에서 가장 높은 산은?
4단계 비교 평가 시트가 'four_way_comparison.csv'에 저장되었습니다.

=== 실험 완료 ===
완전한 RLHF 파이프라인 구축 및 비교 완료


In [5]:
analyze_three_way_results()


3단계 모델 비교 결과 분석

최고 응답 승률:
  기존SFT: 3/12 (25.0%)
  개선SFT: 2/12 (16.7%)
  SFT+RM: 7/12 (58.3%)

유형별 최고 응답:

  factual 유형:
    기존SFT: 1/3 (33.3%)
    개선SFT: 0/3 (0.0%)
    SFT+RM: 2/3 (66.7%)

  explanation 유형:
    기존SFT: 1/3 (33.3%)
    개선SFT: 0/3 (0.0%)
    SFT+RM: 2/3 (66.7%)

  how_to 유형:
    기존SFT: 1/3 (33.3%)
    개선SFT: 1/3 (33.3%)
    SFT+RM: 1/3 (33.3%)

  advice 유형:
    기존SFT: 0/3 (0.0%)
    개선SFT: 1/3 (33.3%)
    SFT+RM: 2/3 (66.7%)

결론:
  SFT+RM가 58.3% 승률로 최고 성능


In [16]:
import numpy as np
from scipy import stats
import time

def load_comparison_models():
    """SFT와 PPO 모델 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    print("모델 로드 중...")
    
    # SFT 모델
    sft_model = AutoModelForCausalLM.from_pretrained("models/improved_sft_v2")
    
    # PPO 모델
    ppo_model = AutoModelForCausalLM.from_pretrained("models/ppo_final")
    
    # RM 모델
    checkpoint = torch.load('models/fixed_rm/reward_model.pt')
    config = GPT2Config.from_dict(checkpoint['config_dict'])
    
    gpt2_model = GPT2Model(config)
    gpt2_model.resize_token_embeddings(len(tokenizer))
    value_head = nn.Linear(config.n_embd, 1)
    rm_model = RewardModel(gpt2_model, value_head)
    
    rm_model.model.load_state_dict(checkpoint['model_state_dict'])
    rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
    
    if torch.cuda.is_available():
        sft_model = sft_model.cuda()
        ppo_model = ppo_model.cuda()
        rm_model = rm_model.cuda()
    
    return sft_model, ppo_model, rm_model, tokenizer

def load_test_questions():
    """코드에서 사용된 핵심 5개 질문"""
    questions = [
        "불고기용 고기 한우에요?",
        "리처드 닉슨이 43대 부통령직을 수행한 년도는?", 
        "시카고 오헤어 국제공항은 어디에 있어?",
        "오늘 미세먼지 어때?",
        "인공지능이 뭐야?"
    ]
    
    return questions

def generate_response(model, tokenizer, question):
    """응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id
        )
    
    generation_time = time.time() - start_time
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    
    return response.strip(), generation_time

def get_rm_score(rm_model, tokenizer, prompt, response):
    """RM 점수 계산"""
    full_text = prompt + response
    inputs = tokenizer.encode(full_text, return_tensors='pt', truncation=True, max_length=512)
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        score = rm_model(inputs).cpu().item()
    
    return score

def run_simple_comparison():
    """간단한 비교 실행"""
    print("=== SFT vs PPO 간단 비교 ===")
    
    # 모델 로드
    sft_model, ppo_model, rm_model, tokenizer = load_comparison_models()
    
    # 질문 로드
    questions = load_test_questions()
    
    # 결과 저장
    results = []
    
    print(f"비교 진행 중... ({len(questions)}개 질문)")
    
    for i, question in enumerate(questions):
        if (i + 1) % 10 == 0:
            print(f"진행률: {i+1}/{len(questions)}")
        
        prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
        
        # SFT 응답
        sft_response, sft_time = generate_response(sft_model, tokenizer, question)
        sft_score = get_rm_score(rm_model, tokenizer, prompt, sft_response)
        
        # PPO 응답
        ppo_response, ppo_time = generate_response(ppo_model, tokenizer, question)
        ppo_score = get_rm_score(rm_model, tokenizer, prompt, ppo_response)
        
        results.append({
            'question': question,
            'sft_response': sft_response,
            'sft_score': sft_score,
            'sft_length': len(sft_response),
            'sft_time': sft_time,
            'ppo_response': ppo_response,
            'ppo_score': ppo_score,
            'ppo_length': len(ppo_response),
            'ppo_time': ppo_time,
            'score_diff': ppo_score - sft_score
        })
    
    return results

def analyze_results(results):
    """결과 분석"""
    print("\n=== 분석 결과 ===")
    
    # 기본 통계
    sft_scores = [r['sft_score'] for r in results]
    ppo_scores = [r['ppo_score'] for r in results]
    score_diffs = [r['score_diff'] for r in results]
    
    sft_lengths = [r['sft_length'] for r in results]
    ppo_lengths = [r['ppo_length'] for r in results]
    
    sft_times = [r['sft_time'] for r in results]
    ppo_times = [r['ppo_time'] for r in results]
    
    print(f"총 질문 수: {len(results)}")
    print(f"\nRM 점수:")
    print(f"  SFT 평균: {np.mean(sft_scores):.4f}")
    print(f"  PPO 평균: {np.mean(ppo_scores):.4f}")
    print(f"  차이: {np.mean(score_diffs):+.4f}")
    
    print(f"\n응답 길이:")
    print(f"  SFT 평균: {np.mean(sft_lengths):.1f}자")
    print(f"  PPO 평균: {np.mean(ppo_lengths):.1f}자")
    
    print(f"\n생성 시간:")
    print(f"  SFT 평균: {np.mean(sft_times):.2f}초")
    print(f"  PPO 평균: {np.mean(ppo_times):.2f}초")
    
    # 승부 결과
    ppo_wins = sum(1 for r in results if r['score_diff'] > 0)
    sft_wins = sum(1 for r in results if r['score_diff'] < 0)
    ties = sum(1 for r in results if r['score_diff'] == 0)
    
    print(f"\n승부 결과:")
    print(f"  PPO 승리: {ppo_wins}회 ({ppo_wins/len(results)*100:.1f}%)")
    print(f"  SFT 승리: {sft_wins}회 ({sft_wins/len(results)*100:.1f}%)")
    print(f"  무승부: {ties}회 ({ties/len(results)*100:.1f}%)")
    
    # 통계적 유의성
    t_stat, p_value = stats.ttest_rel(ppo_scores, sft_scores)
    print(f"\n통계적 검정:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  유의함: {'예' if p_value < 0.05 else '아니오'}")
    
    return {
        'sft_scores': sft_scores,
        'ppo_scores': ppo_scores,
        'sft_lengths': sft_lengths,
        'ppo_lengths': ppo_lengths,
        'ppo_win_rate': ppo_wins/len(results),
        'p_value': p_value
    }

def show_examples(results):
    """대표 사례 보기"""
    print("\n=== 대표 사례 ===")
    
    # 점수 차이로 정렬
    sorted_results = sorted(results, key=lambda x: abs(x['score_diff']), reverse=True)
    
    print("\n1. PPO가 크게 이긴 사례:")
    ppo_big_wins = [r for r in sorted_results if r['score_diff'] > 0][:3]
    for i, r in enumerate(ppo_big_wins, 1):
        print(f"\n{i}. {r['question']}")
        print(f"   점수차: {r['score_diff']:+.4f}")
        print(f"   SFT: {r['sft_response'][:80]}...")
        print(f"   PPO: {r['ppo_response'][:80]}...")
    
    print("\n2. SFT가 크게 이긴 사례:")
    sft_big_wins = [r for r in sorted_results if r['score_diff'] < 0][:3]
    for i, r in enumerate(sft_big_wins, 1):
        print(f"\n{i}. {r['question']}")
        print(f"   점수차: {r['score_diff']:+.4f}")
        print(f"   SFT: {r['sft_response'][:80]}...")
        print(f"   PPO: {r['ppo_response'][:80]}...")

def main_comparison():
    """메인 비교 실행"""
    results = run_simple_comparison()
    stats = analyze_results(results)
    show_examples(results)
    
    # 최종 결론
    print(f"\n=== 최종 결론 ===")
    if stats['ppo_win_rate'] > 0.5:
        winner = "PPO"
        win_rate = stats['ppo_win_rate'] * 100
    else:
        winner = "SFT"
        win_rate = (1 - stats['ppo_win_rate']) * 100
    
    print(f"{winner}가 {win_rate:.1f}% 승률로 우승!")
    print(f"통계적 유의성: {'있음' if stats['p_value'] < 0.05 else '없음'}")
    
    return results

# 실행
if __name__ == "__main__":
    results = main_comparison()

=== SFT vs PPO 간단 비교 ===
모델 로드 중...
비교 진행 중... (5개 질문)

=== 분석 결과 ===
총 질문 수: 5

RM 점수:
  SFT 평균: 0.6991
  PPO 평균: 0.5401
  차이: -0.1590

응답 길이:
  SFT 평균: 173.6자
  PPO 평균: 116.4자

생성 시간:
  SFT 평균: 0.63초
  PPO 평균: 0.45초

승부 결과:
  PPO 승리: 2회 (40.0%)
  SFT 승리: 3회 (60.0%)
  무승부: 0회 (0.0%)

통계적 검정:
  t-statistic: -0.9263
  p-value: 0.4067
  유의함: 아니오

=== 대표 사례 ===

1. PPO가 크게 이긴 사례:

1. 인공지능이 뭐야?
   점수차: +0.2772
   SFT: '저는 인공지능 어시스턴트이기 때문에, 인간의 지능에 대한 개인적인 의견을 제공할 수는 없지만, 인간과 유사한 생각을 가진 분들이 계시면 답변을...
   PPO: '인공지능은 인간의 지식과 경험에 대한 이해와 응용, 그리고 이를 이용하여 새로운 비즈니스 모델과 서비스를 창출하는 기술입니다. 이러한 인공 지...

2. 불고기용 고기 한우에요?
   점수차: +0.1987
   SFT: '저는 인공지능 언어모델이며, 저는 자연어 처리를 하지 않습니다. 하지만, 일반적으로 불고기의 소스와 양념, 그리고 쇠고기 요리에 사용되는 재료...
   PPO: '저는 인공지능 언어모델로써 답변을 제공하는 AI 어시스턴트입니다.\n\n죄송하지만, 저는 식당에서의 고기를 판매하는 가게가 아니므로 해당 식당...

2. SFT가 크게 이긴 사례:

1. 리처드 닉슨이 43대 부통령직을 수행한 년도는?
   점수차: -0.6255
   SFT: '저는 이전에 검색한 데이터에 의해 리처드 닉슨의 47대 부통령직 수행년도가 정확히 파악할 수 없습니다. 추가적인 정보가 제공되면 더 정확한 답...
   PPO: '리처드 닉슨은 41대 부통령직을 수행했

In [17]:
import numpy as np
from scipy import stats
import time

def load_comparison_models():
    """Base와 PPO 모델 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    print("모델 로드 중...")
    
    # Base 모델 (SFT 이전 원본)
    base_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
    
    # PPO 모델 (전체 파이프라인 완료)
    ppo_model = AutoModelForCausalLM.from_pretrained("models/output_3_PPO")
    
    # RM 모델
    checkpoint = torch.load('models/fixed_rm/reward_model.pt')
    config = GPT2Config.from_dict(checkpoint['config_dict'])
    
    gpt2_model = GPT2Model(config)
    gpt2_model.resize_token_embeddings(len(tokenizer))
    value_head = nn.Linear(config.n_embd, 1)
    rm_model = RewardModel(gpt2_model, value_head)
    
    rm_model.model.load_state_dict(checkpoint['model_state_dict'])
    rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
    
    if torch.cuda.is_available():
        base_model = base_model.cuda()
        ppo_model = ppo_model.cuda()
        rm_model = rm_model.cuda()
    
    return base_model, ppo_model, rm_model, tokenizer

def load_test_questions():
    """코드에서 사용된 핵심 5개 질문"""
    questions = [
        "불고기용 고기 한우에요?",
        "리처드 닉슨이 43대 부통령직을 수행한 년도는?", 
        "시카고 오헤어 국제공항은 어디에 있어?",
        "오늘 미세먼지 어때?",
        "인공지능이 뭐야?"
    ]
    
    return questions

def generate_response(model, tokenizer, question):
    """응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id
        )
    
    generation_time = time.time() - start_time
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    
    return response.strip(), generation_time

def get_rm_score(rm_model, tokenizer, prompt, response):
    """RM 점수 계산"""
    full_text = prompt + response
    inputs = tokenizer.encode(full_text, return_tensors='pt', truncation=True, max_length=512)
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        score = rm_model(inputs).cpu().item()
    
    return score

def run_simple_comparison():
    """간단한 비교 실행"""
    print("=== Base vs PPO 간단 비교 ===")
    
    # 모델 로드
    base_model, ppo_model, rm_model, tokenizer = load_comparison_models()
    
    # 질문 로드
    questions = load_test_questions()
    
    # 결과 저장
    results = []
    
    print(f"비교 진행 중... ({len(questions)}개 질문)")
    
    for i, question in enumerate(questions):
        if (i + 1) % 10 == 0:
            print(f"진행률: {i+1}/{len(questions)}")
        
        prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
        
        # Base 모델 응답
        base_response, base_time = generate_response(base_model, tokenizer, question)
        base_score = get_rm_score(rm_model, tokenizer, prompt, base_response)
        
        # PPO 응답
        ppo_response, ppo_time = generate_response(ppo_model, tokenizer, question)
        ppo_score = get_rm_score(rm_model, tokenizer, prompt, ppo_response)
        
        results.append({
            'question': question,
            'base_response': base_response,
            'base_score': base_score,
            'base_length': len(base_response),
            'base_time': base_time,
            'ppo_response': ppo_response,
            'ppo_score': ppo_score,
            'ppo_length': len(ppo_response),
            'ppo_time': ppo_time,
            'score_diff': ppo_score - base_score
        })
    
    return results

def analyze_results(results):
    """결과 분석"""
    print("\n=== 분석 결과 ===")
    
    # 기본 통계
    base_scores = [r['base_score'] for r in results]
    ppo_scores = [r['ppo_score'] for r in results]
    score_diffs = [r['score_diff'] for r in results]
    
    base_lengths = [r['base_length'] for r in results]
    ppo_lengths = [r['ppo_length'] for r in results]
    
    base_times = [r['base_time'] for r in results]
    ppo_times = [r['ppo_time'] for r in results]
    
    print(f"총 질문 수: {len(results)}")
    print(f"\nRM 점수:")
    print(f"  Base 평균: {np.mean(base_scores):.4f}")
    print(f"  PPO 평균: {np.mean(ppo_scores):.4f}")
    print(f"  차이: {np.mean(score_diffs):+.4f}")
    
    print(f"\n응답 길이:")
    print(f"  Base 평균: {np.mean(base_lengths):.1f}자")
    print(f"  PPO 평균: {np.mean(ppo_lengths):.1f}자")
    
    print(f"\n생성 시간:")
    print(f"  Base 평균: {np.mean(base_times):.2f}초")
    print(f"  PPO 평균: {np.mean(ppo_times):.2f}초")
    
    # 승부 결과
    ppo_wins = sum(1 for r in results if r['score_diff'] > 0)
    base_wins = sum(1 for r in results if r['score_diff'] < 0)
    ties = sum(1 for r in results if r['score_diff'] == 0)
    
    print(f"\n승부 결과:")
    print(f"  PPO 승리: {ppo_wins}회 ({ppo_wins/len(results)*100:.1f}%)")
    print(f"  Base 승리: {base_wins}회 ({base_wins/len(results)*100:.1f}%)")
    print(f"  무승부: {ties}회 ({ties/len(results)*100:.1f}%)")
    
    # 통계적 유의성
    t_stat, p_value = stats.ttest_rel(ppo_scores, base_scores)
    print(f"\n통계적 검정:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  유의함: {'예' if p_value < 0.05 else '아니오'}")
    
    return {
        'base_scores': base_scores,
        'ppo_scores': ppo_scores,
        'base_lengths': base_lengths,
        'ppo_lengths': ppo_lengths,
        'ppo_win_rate': ppo_wins/len(results),
        'p_value': p_value
    }

def show_examples(results):
    """5개 질문 각각의 답변 예시"""
    print("\n=== 각 질문별 응답 예시 ===")
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. 질문: {result['question']}")
        print(f"   점수차: {result['score_diff']:+.4f}")
        print(f"   Base 응답: {result['base_response']}")
        print(f"   PPO 응답: {result['ppo_response']}")
        print("-" * 80)

def qualitative_analysis(results):
    """정성 분석"""
    print("\n=== 정성 분석 ===")
    
    total_questions = len(results)
    ppo_wins = sum(1 for r in results if r['score_diff'] > 0)
    base_wins = sum(1 for r in results if r['score_diff'] < 0)
    
    print(f"전반적 성능:")
    print(f"  PPO가 우세한 질문: {ppo_wins}/{total_questions}")
    print(f"  Base가 우세한 질문: {base_wins}/{total_questions}")
    
    # 응답 품질 특성 분석
    base_lengths = [r['base_length'] for r in results]
    ppo_lengths = [r['ppo_length'] for r in results]
    
    print(f"\n응답 특성:")
    print(f"  Base 모델 - 평균 {np.mean(base_lengths):.1f}자")
    if np.mean(base_lengths) < 50:
        print(f"    -> 매우 짧은 응답 경향")
    elif np.mean(base_lengths) < 100:
        print(f"    -> 간결한 응답 경향") 
    else:
        print(f"    -> 상세한 응답 경향")
        
    print(f"  PPO 모델 - 평균 {np.mean(ppo_lengths):.1f}자")
    if np.mean(ppo_lengths) < 50:
        print(f"    -> 매우 짧은 응답 경향")
    elif np.mean(ppo_lengths) < 100:
        print(f"    -> 간결한 응답 경향")
    else:
        print(f"    -> 상세한 응답 경향")
    
    # RM 점수 분석
    base_scores = [r['base_score'] for r in results]
    ppo_scores = [r['ppo_score'] for r in results]
    
    print(f"\nRM 점수 분석:")
    print(f"  Base 점수 범위: {min(base_scores):.3f} ~ {max(base_scores):.3f}")
    print(f"  PPO 점수 범위: {min(ppo_scores):.3f} ~ {max(ppo_scores):.3f}")
    
    avg_improvement = np.mean([r['score_diff'] for r in results])
    if avg_improvement > 0.1:
        print(f"  -> PPO가 Base 대비 상당한 개선 ({avg_improvement:+.3f})")
    elif avg_improvement > 0:
        print(f"  -> PPO가 Base 대비 약간 개선 ({avg_improvement:+.3f})")
    elif avg_improvement > -0.1:
        print(f"  -> 두 모델 성능이 비슷함 ({avg_improvement:+.3f})")
    else:
        print(f"  -> Base가 PPO보다 우수함 ({avg_improvement:+.3f})")

def main_comparison():
    """메인 비교 실행"""
    results = run_simple_comparison()
    stats = analyze_results(results)
    qualitative_analysis(results)
    show_examples(results)
    
    # 최종 결론
    print(f"\n=== 최종 결론 ===")
    if stats['ppo_win_rate'] > 0.5:
        winner = "PPO"
        win_rate = stats['ppo_win_rate'] * 100
    else:
        winner = "Base"
        win_rate = (1 - stats['ppo_win_rate']) * 100
    
    print(f"{winner}가 {win_rate:.1f}% 승률로 우승!")
    print(f"통계적 유의성: {'있음' if stats['p_value'] < 0.05 else '없음'}")
    
    return results

# 실행
if __name__ == "__main__":
    results = main_comparison()

=== Base vs PPO 간단 비교 ===
모델 로드 중...


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

비교 진행 중... (5개 질문)

=== 분석 결과 ===
총 질문 수: 5

RM 점수:
  Base 평균: -0.0139
  PPO 평균: 0.3348
  차이: +0.3487

응답 길이:
  Base 평균: 209.2자
  PPO 평균: 106.8자

생성 시간:
  Base 평균: 1.24초
  PPO 평균: 0.40초

승부 결과:
  PPO 승리: 4회 (80.0%)
  Base 승리: 1회 (20.0%)
  무승부: 0회 (0.0%)

통계적 검정:
  t-statistic: 1.4333
  p-value: 0.2251
  유의함: 아니오

=== 정성 분석 ===
전반적 성능:
  PPO가 우세한 질문: 4/5
  Base가 우세한 질문: 1/5

응답 특성:
  Base 모델 - 평균 209.2자
    -> 상세한 응답 경향
  PPO 모델 - 평균 106.8자
    -> 상세한 응답 경향

RM 점수 분석:
  Base 점수 범위: -0.465 ~ 0.720
  PPO 점수 범위: -0.328 ~ 0.822
  -> PPO가 Base 대비 상당한 개선 (+0.349)

=== 각 질문별 응답 예시 ===

1. 질문: 불고기용 고기 한우에요?
   점수차: +0.8337
   Base 응답: #Repost (경고):
#Research(위험):
#selfie(위기관리)’:
#daily #like4like #instadog #신혼여행 가기로 했습니다!
아무래도 제가 신혼집일텐데
가까운 곳을 찾아다니다
보니 #서울숲 까지~
내 집처럼 편안하고 안락하게
입주할 수 있는 곳이니만큼 좋은 것 같아요!
역시 서울숲은...
   PPO 응답: '저는 AI 어시스턴트이므로 불고기의 종류, 양, 크기, 부위 등 정확한 정보를 알지 못합니다. 하지만 일반적으로 쇠고기를 먹을 때 사용되는 양념은 고기를 소금에 찍어 먹는 것이 일반적입니다. 또한, 돼지고기는 볶음밥이나 국물을 사용하여 맛을 조절하기도 합니다.
---------------------------

In [18]:
import numpy as np
from scipy import stats
import time

def load_comparison_models():
    """SFT와 PPO 모델 로드"""
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    print("모델 로드 중...")
    
    # SFT 모델
    sft_model = AutoModelForCausalLM.from_pretrained("models/output_1_SFT")
    
    # PPO 모델
    ppo_model = AutoModelForCausalLM.from_pretrained("models/output_3_PPO")
    
    # RM 모델
    checkpoint = torch.load('models/fixed_rm/reward_model.pt')
    config = GPT2Config.from_dict(checkpoint['config_dict'])
    
    gpt2_model = GPT2Model(config)
    gpt2_model.resize_token_embeddings(len(tokenizer))
    value_head = nn.Linear(config.n_embd, 1)
    rm_model = RewardModel(gpt2_model, value_head)
    
    rm_model.model.load_state_dict(checkpoint['model_state_dict'])
    rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
    
    if torch.cuda.is_available():
        sft_model = sft_model.cuda()
        ppo_model = ppo_model.cuda()
        rm_model = rm_model.cuda()
    
    return sft_model, ppo_model, rm_model, tokenizer

def load_test_questions():
    """코드에서 사용된 핵심 5개 질문"""
    questions = [
        "불고기용 고기 한우에요?",
        "리처드 닉슨이 43대 부통령직을 수행한 년도는?", 
        "시카고 오헤어 국제공항은 어디에 있어?",
        "오늘 미세먼지 어때?",
        "인공지능이 뭐야?"
    ]
    
    return questions

def generate_response(model, tokenizer, question):
    """응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id
        )
    
    generation_time = time.time() - start_time
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    
    return response.strip(), generation_time

def get_rm_score(rm_model, tokenizer, prompt, response):
    """RM 점수 계산"""
    full_text = prompt + response
    inputs = tokenizer.encode(full_text, return_tensors='pt', truncation=True, max_length=512)
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        score = rm_model(inputs).cpu().item()
    
    return score

def run_simple_comparison():
    """간단한 비교 실행"""
    print("=== SFT vs PPO 간단 비교 ===")
    
    # 모델 로드
    sft_model, ppo_model, rm_model, tokenizer = load_comparison_models()
    
    # 질문 로드
    questions = load_test_questions()
    
    # 결과 저장
    results = []
    
    print(f"비교 진행 중... ({len(questions)}개 질문)")
    
    for i, question in enumerate(questions):
        if (i + 1) % 10 == 0:
            print(f"진행률: {i+1}/{len(questions)}")
        
        prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
        
        # SFT 응답
        sft_response, sft_time = generate_response(sft_model, tokenizer, question)
        sft_score = get_rm_score(rm_model, tokenizer, prompt, sft_response)
        
        # PPO 응답
        ppo_response, ppo_time = generate_response(ppo_model, tokenizer, question)
        ppo_score = get_rm_score(rm_model, tokenizer, prompt, ppo_response)
        
        results.append({
            'question': question,
            'sft_response': sft_response,
            'sft_score': sft_score,
            'sft_length': len(sft_response),
            'sft_time': sft_time,
            'ppo_response': ppo_response,
            'ppo_score': ppo_score,
            'ppo_length': len(ppo_response),
            'ppo_time': ppo_time,
            'score_diff': ppo_score - sft_score
        })
    
    return results

def analyze_results(results):
    """결과 분석"""
    print("\n=== 분석 결과 ===")
    
    # 기본 통계
    sft_scores = [r['sft_score'] for r in results]
    ppo_scores = [r['ppo_score'] for r in results]
    score_diffs = [r['score_diff'] for r in results]
    
    sft_lengths = [r['sft_length'] for r in results]
    ppo_lengths = [r['ppo_length'] for r in results]
    
    sft_times = [r['sft_time'] for r in results]
    ppo_times = [r['ppo_time'] for r in results]
    
    print(f"총 질문 수: {len(results)}")
    print(f"\nRM 점수:")
    print(f"  SFT 평균: {np.mean(sft_scores):.4f}")
    print(f"  PPO 평균: {np.mean(ppo_scores):.4f}")
    print(f"  차이: {np.mean(score_diffs):+.4f}")
    
    print(f"\n응답 길이:")
    print(f"  SFT 평균: {np.mean(sft_lengths):.1f}자")
    print(f"  PPO 평균: {np.mean(ppo_lengths):.1f}자")
    
    print(f"\n생성 시간:")
    print(f"  SFT 평균: {np.mean(sft_times):.2f}초")
    print(f"  PPO 평균: {np.mean(ppo_times):.2f}초")
    
    # 승부 결과
    ppo_wins = sum(1 for r in results if r['score_diff'] > 0)
    sft_wins = sum(1 for r in results if r['score_diff'] < 0)
    ties = sum(1 for r in results if r['score_diff'] == 0)
    
    print(f"\n승부 결과:")
    print(f"  PPO 승리: {ppo_wins}회 ({ppo_wins/len(results)*100:.1f}%)")
    print(f"  SFT 승리: {sft_wins}회 ({sft_wins/len(results)*100:.1f}%)")
    print(f"  무승부: {ties}회 ({ties/len(results)*100:.1f}%)")
    
    return {
        'sft_scores': sft_scores,
        'ppo_scores': ppo_scores,
        'sft_lengths': sft_lengths,
        'ppo_lengths': ppo_lengths,
        'ppo_win_rate': ppo_wins/len(results)
    }

def qualitative_analysis(results):
    """정성 분석"""
    print("\n=== 정성 분석 ===")
    
    total_questions = len(results)
    ppo_wins = sum(1 for r in results if r['score_diff'] > 0)
    sft_wins = sum(1 for r in results if r['score_diff'] < 0)
    
    print(f"전반적 성능:")
    print(f"  PPO가 우세한 질문: {ppo_wins}/{total_questions}")
    print(f"  SFT가 우세한 질문: {sft_wins}/{total_questions}")
    
    # 응답 품질 특성 분석
    sft_lengths = [r['sft_length'] for r in results]
    ppo_lengths = [r['ppo_length'] for r in results]
    
    print(f"\n응답 특성:")
    print(f"  SFT 모델 - 평균 {np.mean(sft_lengths):.1f}자")
    if np.mean(sft_lengths) < 50:
        print(f"    -> 매우 짧은 응답 경향")
    elif np.mean(sft_lengths) < 100:
        print(f"    -> 간결한 응답 경향") 
    else:
        print(f"    -> 상세한 응답 경향")
        
    print(f"  PPO 모델 - 평균 {np.mean(ppo_lengths):.1f}자")
    if np.mean(ppo_lengths) < 50:
        print(f"    -> 매우 짧은 응답 경향")
    elif np.mean(ppo_lengths) < 100:
        print(f"    -> 간결한 응답 경향")
    else:
        print(f"    -> 상세한 응답 경향")
    
    # RM 점수 분석
    sft_scores = [r['sft_score'] for r in results]
    ppo_scores = [r['ppo_score'] for r in results]
    
    print(f"\nRM 점수 분석:")
    print(f"  SFT 점수 범위: {min(sft_scores):.3f} ~ {max(sft_scores):.3f}")
    print(f"  PPO 점수 범위: {min(ppo_scores):.3f} ~ {max(ppo_scores):.3f}")
    
    avg_improvement = np.mean([r['score_diff'] for r in results])
    if avg_improvement > 0.1:
        print(f"  -> PPO가 SFT 대비 상당한 개선 ({avg_improvement:+.3f})")
    elif avg_improvement > 0:
        print(f"  -> PPO가 SFT 대비 약간 개선 ({avg_improvement:+.3f})")
    elif avg_improvement > -0.1:
        print(f"  -> 두 모델 성능이 비슷함 ({avg_improvement:+.3f})")
    else:
        print(f"  -> SFT가 PPO보다 우수함 ({avg_improvement:+.3f})")

def show_examples(results):
    """5개 질문 각각의 답변 예시"""
    print("\n=== 각 질문별 응답 예시 ===")
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. 질문: {result['question']}")
        print(f"   점수차: {result['score_diff']:+.4f}")
        print(f"   SFT 응답: {result['sft_response']}")
        print(f"   PPO 응답: {result['ppo_response']}")
        print("-" * 80)

def main_comparison():
    """메인 비교 실행"""
    results = run_simple_comparison()
    stats = analyze_results(results)
    qualitative_analysis(results)
    show_examples(results)
    
    # 최종 결론
    print(f"\n=== 최종 결론 ===")
    if stats['ppo_win_rate'] > 0.5:
        winner = "PPO"
        win_rate = stats['ppo_win_rate'] * 100
    else:
        winner = "SFT"
        win_rate = (1 - stats['ppo_win_rate']) * 100
    
    print(f"{winner}가 {win_rate:.1f}% 승률로 우승!")
    
    return results

# 실행
if __name__ == "__main__":
    results = main_comparison()

=== SFT vs PPO 간단 비교 ===
모델 로드 중...
비교 진행 중... (5개 질문)

=== 분석 결과 ===
총 질문 수: 5

RM 점수:
  SFT 평균: 0.4868
  PPO 평균: 0.4089
  차이: -0.0779

응답 길이:
  SFT 평균: 110.2자
  PPO 평균: 100.4자

생성 시간:
  SFT 평균: 0.44초
  PPO 평균: 0.38초

승부 결과:
  PPO 승리: 2회 (40.0%)
  SFT 승리: 3회 (60.0%)
  무승부: 0회 (0.0%)

=== 정성 분석 ===
전반적 성능:
  PPO가 우세한 질문: 2/5
  SFT가 우세한 질문: 3/5

응답 특성:
  SFT 모델 - 평균 110.2자
    -> 상세한 응답 경향
  PPO 모델 - 평균 100.4자
    -> 상세한 응답 경향

RM 점수 분석:
  SFT 점수 범위: 0.136 ~ 0.952
  PPO 점수 범위: 0.011 ~ 0.781
  -> 두 모델 성능이 비슷함 (-0.078)

=== 각 질문별 응답 예시 ===

1. 질문: 불고기용 고기 한우에요?
   점수차: -0.2100
   SFT 응답: '죄송하지만, 저는 인공지능 언어 모델로써 직접 고기를 다루지는 않습니다. 하지만, 만약 불고기를 사용하는 경우, 해당 음식점의 메뉴판을 확인해보시는 것이 좋겠습니다!
   PPO 응답: '저는 인공지능 언어 모델로써 고기를 먹을 수 없습니다. 하지만 보통 불고기는 다양한 종류의 양념과 양념을 사용하여 맛을 내는 음식입니다. 또한 식당에서 제공되는 불고기 용 고기의 맛은 개인마다 다를 수 있습니다.
--------------------------------------------------------------------------------

2. 질문: 리처드 닉슨이 43대 부통령직을 수행한 년도는?
   점수차: -0.3346
   SFT 응답: '리처드 닉슨은 39대 부통령직을 수행했습니다.
   PPO 응답: '리

In [23]:
import torch
import torch.nn as nn
import numpy as np
import time
import random
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from chatgpt.models.base import RewardModel

def load_random_questions():
    """PPO 데이터에서 랜덤 5개 질문 선택"""
    print("PPO 데이터에서 랜덤 질문 로딩 중...")
    
    with open('KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as f:
        data = json.load(f)
        all_questions = [item['prompt'] for item in data]
    
    random.seed(42)
    selected = random.sample(all_questions, 5)
    
    print(f"전체 {len(all_questions)}개 중 5개 선택:")
    for i, q in enumerate(selected, 1):
        print(f"  {i}. {q}")
    
    return selected

def load_models():
    """SFT, PPO, RM 모델 로드"""
    print("모델들 로딩 중...")
    
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", model_max_length=1024
    )
    
    # SFT 모델
    sft_model = AutoModelForCausalLM.from_pretrained("models/output_1_SFT")
    
    # PPO 모델  
    ppo_model = AutoModelForCausalLM.from_pretrained("models/output_3_PPO")
    
    # RM 모델
    checkpoint = torch.load('models/fixed_rm/reward_model.pt')
    config = GPT2Config.from_dict(checkpoint['config_dict'])
    
    gpt2_model = GPT2Model(config)
    gpt2_model.resize_token_embeddings(len(tokenizer))
    value_head = nn.Linear(config.n_embd, 1)
    rm_model = RewardModel(gpt2_model, value_head)
    
    rm_model.model.load_state_dict(checkpoint['model_state_dict'])
    rm_model.value_head.load_state_dict(checkpoint['value_head_state_dict'])
    
    if torch.cuda.is_available():
        sft_model = sft_model.cuda()
        ppo_model = ppo_model.cuda()  
        rm_model = rm_model.cuda()
    
    return sft_model, ppo_model, rm_model, tokenizer

def generate_response(model, tokenizer, question):
    """모델로 응답 생성"""
    prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 120,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id
        )
    
    gen_time = time.time() - start_time
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    
    return response.strip(), gen_time

def get_reward_score(rm_model, tokenizer, prompt, response):
    """RM으로 점수 계산"""
    full_text = prompt + response
    inputs = tokenizer.encode(full_text, return_tensors='pt', truncation=True, max_length=512)
    
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    with torch.no_grad():
        score = rm_model(inputs).cpu().item()
    
    return score

def run_comparison():
    """SFT vs PPO 비교 실행"""
    print("=== SFT vs PPO 비교 시작 ===")
    
    # 데이터와 모델 로드
    questions = load_random_questions()
    sft_model, ppo_model, rm_model, tokenizer = load_models()
    
    results = []
    
    print("\n응답 생성 중...")
    for i, question in enumerate(questions, 1):
        print(f"처리 중 {i}/5: {question[:30]}...")
        
        prompt = f"### Instruction(명령어):\n{question}\n\n### Response(응답):"
        
        # SFT 응답
        sft_response, sft_time = generate_response(sft_model, tokenizer, question)
        sft_score = get_reward_score(rm_model, tokenizer, prompt, sft_response)
        
        # PPO 응답
        ppo_response, ppo_time = generate_response(ppo_model, tokenizer, question)
        ppo_score = get_reward_score(rm_model, tokenizer, prompt, ppo_response)
        
        results.append({
            'question': question,
            'sft_response': sft_response,
            'sft_score': sft_score,
            'sft_time': sft_time,
            'ppo_response': ppo_response,
            'ppo_score': ppo_score,
            'ppo_time': ppo_time,
            'score_diff': ppo_score - sft_score
        })
    
    return results

def analyze_results(results):
    """정량 분석"""
    print("\n=== 정량 분석 ===")
    
    sft_scores = [r['sft_score'] for r in results]
    ppo_scores = [r['ppo_score'] for r in results]
    score_diffs = [r['score_diff'] for r in results]
    
    sft_lengths = [len(r['sft_response']) for r in results]
    ppo_lengths = [len(r['ppo_response']) for r in results]
    
    sft_times = [r['sft_time'] for r in results]
    ppo_times = [r['ppo_time'] for r in results]
    
    print(f"RM 점수:")
    print(f"  SFT 평균: {np.mean(sft_scores):.4f}")
    print(f"  PPO 평균: {np.mean(ppo_scores):.4f}")
    print(f"  평균 차이: {np.mean(score_diffs):+.4f}")
    
    print(f"\n응답 길이:")
    print(f"  SFT 평균: {np.mean(sft_lengths):.1f}자")
    print(f"  PPO 평균: {np.mean(ppo_lengths):.1f}자")
    
    print(f"\n생성 시간:")
    print(f"  SFT 평균: {np.mean(sft_times):.2f}초")
    print(f"  PPO 평균: {np.mean(ppo_times):.2f}초")
    
    # 승부 결과
    ppo_wins = sum(1 for diff in score_diffs if diff > 0)
    sft_wins = sum(1 for diff in score_diffs if diff < 0)
    ties = sum(1 for diff in score_diffs if diff == 0)
    
    print(f"\n승부 결과:")
    print(f"  PPO 승: {ppo_wins}회 ({ppo_wins/5*100:.0f}%)")
    print(f"  SFT 승: {sft_wins}회 ({sft_wins/5*100:.0f}%)")
    print(f"  무승부: {ties}회 ({ties/5*100:.0f}%)")
    
    return ppo_wins > sft_wins

def qualitative_analysis(results):
    """정성 분석"""
    print("\n=== 정성 분석 ===")
    
    sft_lengths = [len(r['sft_response']) for r in results]
    ppo_lengths = [len(r['ppo_response']) for r in results]
    
    print("응답 특성:")
    sft_avg = np.mean(sft_lengths)
    ppo_avg = np.mean(ppo_lengths)
    
    print(f"  SFT: 평균 {sft_avg:.0f}자 - ", end="")
    if sft_avg < 50:
        print("매우 간결한 응답")
    elif sft_avg < 100:
        print("간결한 응답")
    else:
        print("상세한 응답")
    
    print(f"  PPO: 평균 {ppo_avg:.0f}자 - ", end="")
    if ppo_avg < 50:
        print("매우 간결한 응답")
    elif ppo_avg < 100:
        print("간결한 응답")
    else:
        print("상세한 응답")
    
    # 점수 개선도
    avg_improvement = np.mean([r['score_diff'] for r in results])
    print(f"\n성능 개선:")
    if avg_improvement > 0.1:
        print(f"  PPO가 SFT 대비 크게 개선됨 ({avg_improvement:+.3f})")
    elif avg_improvement > 0:
        print(f"  PPO가 SFT 대비 약간 개선됨 ({avg_improvement:+.3f})")
    elif avg_improvement > -0.1:
        print(f"  두 모델 성능이 비슷함 ({avg_improvement:+.3f})")
    else:
        print(f"  SFT가 PPO보다 우수함 ({avg_improvement:+.3f})")

def show_examples(results):
    """응답 예시 출력"""
    print("\n=== 응답 예시 ===")
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. {result['question']}")
        print(f"   점수차: {result['score_diff']:+.4f}")
        print(f"   SFT: {result['sft_response']}")
        print(f"   PPO: {result['ppo_response']}")
        print("-" * 60)

def main():
    """메인 실행"""
    results = run_comparison()
    ppo_won = analyze_results(results)
    qualitative_analysis(results)
    show_examples(results)
    
    print(f"\n=== 최종 결과 ===")
    winner = "PPO" if ppo_won else "SFT"
    print(f"{winner} 모델이 더 우수한 성능을 보였습니다!")
    
    return results

# 실행
if __name__ == "__main__":
    results = main()

=== SFT vs PPO 비교 시작 ===
PPO 데이터에서 랜덤 질문 로딩 중...
전체 12000개 중 5개 선택:
  1. 붙잡으려 말 것!
  2. 귀찮으면 썸 끝인가
  3. 반기문과 경쟁하던 샤시 타루르의 국적은?
  4. 남자친구 말고 다른 남자도 궁금해.
  5. 네 그걸로 주시고 적립되나요?
모델들 로딩 중...

응답 생성 중...
처리 중 1/5: 붙잡으려 말 것!...
처리 중 2/5: 귀찮으면 썸 끝인가...
처리 중 3/5: 반기문과 경쟁하던 샤시 타루르의 국적은?...
처리 중 4/5: 남자친구 말고 다른 남자도 궁금해....
처리 중 5/5: 네 그걸로 주시고 적립되나요?...

=== 정량 분석 ===
RM 점수:
  SFT 평균: 0.3103
  PPO 평균: 0.2480
  평균 차이: -0.0622

응답 길이:
  SFT 평균: 141.0자
  PPO 평균: 137.8자

생성 시간:
  SFT 평균: 0.53초
  PPO 평균: 0.51초

승부 결과:
  PPO 승: 2회 (40%)
  SFT 승: 3회 (60%)
  무승부: 0회 (0%)

=== 정성 분석 ===
응답 특성:
  SFT: 평균 141자 - 상세한 응답
  PPO: 평균 138자 - 상세한 응답

성능 개선:
  두 모델 성능이 비슷함 (-0.062)

=== 응답 예시 ===

1. 붙잡으려 말 것!
   점수차: +0.1472
   SFT: '저도 당신을 좋아합니다. 하지만 만약 어떤 일이 있었다면, 저는 그 일에 대해 다시 한번 생각해보겠습니다. 예를 들어, 무엇을 도와드릴까요?
   PPO: '저는 AI 언어 모델입니다. 그러나, 어떤 종류의 붙박이 짓이든 그것은 당신의 삶에서 중요한 요소 중 하나입니다. 따라서, 당신이 그 사람과 대화를 나누어보고, 그것이 당신의 삶에 영향을 미칠 수 있도록 노력하십시오.
------------------------------------------------------------

2. 귀찮으면 썸 끝인가


# 회고

### 실험 진행 방향
팀 베이스라인 모델에 추가적인 SFT -> RM -> PPO 진행
### 실험 결과
기존 모델과의 PPO모델 
RM 점수 분석:
  Base 점수 범위: -0.465 ~ 0.720
  PPO 점수 범위: -0.328 ~ 0.822
  -> PPO가 Base 대비 상당한 개선 (+0.349)
SFT모델과 PPO 모델
성능 개선:
RM 점수:
  SFT 평균: 0.3103
  PPO 평균: 0.2480
  평균 차이: -0.0622
  두 모델 성능이 비슷함 (-0.062)
## SFT 단계에서 가장 큰 성능 향상

### 느낀점
우선 RM 자체를 내가 직접 하다보니 나라는 사람의 편향이 모델에 반영이 되면서 객관성도 떨어지고 성능도 좋아지지않았다는걸 알게되었다.
RLHF의 각 단계가 실제로 어떻게 작동하는지 직접 경험하고 이해할 수 있는 좋은 경험이 되었던 것 같고, 
팀내에서 새롭게 생성한 데이터를 추가해보고 학습을 좀 더 시켜보는 식으로 추가실험을 해 볼 경우 어느정도 성능이 개선될 것 같아서 추가실험을 해 볼 예정이다.